In [208]:
import shap
print(shap.__version__)

0.45.1


In [209]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [210]:
# print elasticsearch search package version
import elasticsearch
print(elasticsearch.__version__)

(8, 13, 2)


In [211]:
from elasticsearch import Elasticsearch, helpers
import time

# 1. Connect to ES (edit the host/port as needed)
es = Elasticsearch("http://localhost:9200")
index_name = "bm25-demo"

# 2. Delete index if it exists (for repeatable demo)
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# 3. Create index with BM25 (standard analyzer by default)
mapping = {

    "mappings": {
        "properties": {
            "content": {
                "type": "text",  # Uses BM25 and standard analyzer by default
            }
        }
    }
}
es.indices.create(index=index_name, body=mapping)
print(f"Created index '{index_name}'.")

# 4. Index a few documents
docs = [
    {"content": "The stock market saw gains today in the finance sector."},
    {"content": "Technology advances are happening at a rapid pace."},
    {"content": "Healthy eating and exercise improve overall health."},
    {"content": "Weather forecasts predict rain this week."},
    {"content": "Financial experts recommend saving early for retirement."},
]
actions = [{"_index": index_name, "_source": doc} for doc in docs]
helpers.bulk(es, actions)
print("Indexed documents.")
time.sleep(1)  # Let ES index the docs



import sys
sys.path.append('/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/')
# import rankingSHAP_test_class_cutomized
from rankingSHAP_test_class_cutomized import extract_es_features_optimized, extract_features_from_documents 
# from rankingSHAP_test_class import extract_es_features_optimized
# ----- Usage -----
es = Elasticsearch("http://localhost:9200")
index_name = "bm25-demo"

# Extract features using the optimized method
features = extract_es_features_optimized(es, index_name, field="content")
print(features)
print(len(features))








Created index 'bm25-demo'.
Indexed documents.
['a', 'advances', 'and', 'are', 'at', 'early', 'eating', 'exercise', 'experts', 'finance', 'financial', 'for', 'forecasts', 'gains', 'happening', 'health', 'healthy', 'improve', 'in', 'market', 'overall', 'pace', 'predict', 'rain', 'rapid', 'recommend', 'retirement', 'saving', 'saw', 'sector', 'stock', 'technology', 'the', 'this', 'today', 'weather', 'week']
37


In [212]:
import random

documents = [
    "The stock market saw gains today in the finance sector.",
    "Technology advances are happening at a rapid pace.",
    "Healthy eating and exercise improve overall health.",
    "Weather forecasts predict rain this week.",
    "Financial experts recommend saving early for retirement.",
    "Machine learning models require large datasets for training.",
    "The weather today is sunny and warm.",
    "Exercise and diet are key to maintaining good health."
]

finance_variations = [
    "The stock market saw slight losses today in the tech sector.",
    "Investors showed optimism in the banking industry this morning.",
    "Financial analysts expect interest rates to rise next quarter.",
    "The bond market remained steady amid economic uncertainty."
]

tech_variations = [
    "New breakthroughs in AI are transforming industries rapidly.",
    "Advances in robotics and automation continue to accelerate.",
    "Quantum computing shows promise for solving complex problems.",
    "Tech startups are innovating at an unprecedented pace."
]

health_variations = [
    "Regular exercise and balanced nutrition enhance physical fitness.",
    "Maintaining a healthy diet reduces risk of chronic diseases.",
    "Sleep and hydration play a key role in wellness.",
    "Mental health awareness is becoming increasingly important."
]

weather_variations = [
    "Meteorologists predict heavy snowfall this weekend.",
    "A heatwave is expected to hit the southern region soon.",
    "Storm warnings have been issued for coastal areas.",
    "Temperatures are expected to remain mild throughout the week."
]

ml_variations = [
    "Deep learning models rely heavily on high-quality labeled data.",
    "AI models often require fine-tuning to achieve optimal accuracy.",
    "Model performance depends greatly on data preprocessing.",
    "Training neural networks demands substantial computational resources."
]

all_templates = finance_variations + tech_variations + health_variations + weather_variations + ml_variations + documents

# random combination to make 100 records
synthetic_documents = [random.choice(all_templates) for _ in range(100)]
print(len(synthetic_documents))
# for i, doc in enumerate(synthetic_documents[:10], 1):  # print first 10 for preview
#     print(f"{i}. {doc}")

# # print(f"\nTotal records generated: {len(synthetic_documents)}")


100


In [ ]:
from elasticsearch import Elasticsearch, helpers
from typing import List, Dict, Any, Optional
import uuid

class ElasticsearchSparseSearchModel:
    """
    A simple Elasticsearch sparse search model with scikit-learn-like interface.
    Uses BM25 scoring for sparse retrieval.
    """
    
    def __init__(self, es_host: str = "http://localhost:9200", index_name: Optional[str] = None):
        """
        Initialize the Elasticsearch sparse search model.
        
        Args:
            es_host (str): Elasticsearch host URL
            index_name (str, optional): Index name. If None, generates a random one.
        """
        self.es_host = es_host
        self.index_name = index_name or f"sparse_search_{uuid.uuid4().hex[:8]}"
        self.es = None
        self.is_fitted = False
        self.field_name = "content"
        
    def _connect(self):
        """Establish connection to Elasticsearch."""
        if self.es is None:
            try:
                self.es = Elasticsearch(self.es_host)
                if not self.es.ping():
                    raise ConnectionError("Cannot connect to Elasticsearch")
                print(f"Connected to Elasticsearch at {self.es_host}")
            except Exception as e:
                raise ConnectionError(f"Failed to connect to Elasticsearch: {e}")
    
    def fit(self, documents: List[str]) -> 'ElasticsearchSparseSearchModel':
        """
        Fit the model by indexing documents into Elasticsearch.
        
        Args:
            documents (List[str]): List of document texts to index
            
        Returns:
            self: Returns the instance for method chaining
        """
        self._connect()
        
        # Delete existing index if it exists
        if self.es.indices.exists(index=self.index_name):
            self.es.indices.delete(index=self.index_name)
            print(f"Deleted existing index '{self.index_name}'")
        
        # Create index with BM25 settings
        mapping = {
            "mappings": {
                "properties": {
                    self.field_name: {
                        "type": "text",
                        "analyzer": "standard"  # Uses BM25 by default
                    }
                }
            }
        }
        
        try:
            self.es.indices.create(index=self.index_name, **mapping)
            print(f"Created index '{self.index_name}'")
        except Exception as e:
            raise RuntimeError(f"Failed to create index: {e}")
        
        # Index documents
        actions = []
        for i, doc in enumerate(documents):
            actions.append({
                "_index": self.index_name,
                "_id": i,
                "_source": {self.field_name: doc}
            })
        
        try:
            helpers.bulk(self.es, actions)
            self.es.indices.refresh(index=self.index_name)
            print(f"Indexed {len(documents)} documents")
            self.is_fitted = True
        except Exception as e:
            raise RuntimeError(f"Failed to index documents: {e}")
        
        return self
    
    def predict(self, queries: List[str], top_k: int = 5) -> List[List[Dict[str, Any]]]:
        """
        Perform sparse search predictions for given queries.
        
        Args:
            queries (List[str]): List of query strings
            top_k (int): Number of top results to return per query
            
        Returns:
            List[List[Dict]]: For each query, returns list of top_k results.
                            Each result contains: {'doc_id', 'score', 'content'}
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before making predictions. Call .fit() first.")
        
        results = []
        
        for query in queries:
            # Elasticsearch query using BM25
            search_body = {
                "query": {
                    "match": {
                        self.field_name: query
                    }
                },
                "size": top_k,
                "_source": [self.field_name]
            }
            
            try:
                response = self.es.search(index=self.index_name, body=search_body)
                
                query_results = []
                for hit in response['hits']['hits']:
                    result = {
                        'doc_id': hit['_id'],
                        'score': hit['_score'],
                        'content': hit['_source'][self.field_name]
                    }
                    query_results.append(result)
                
                results.append(query_results)
                
            except Exception as e:
                print(f"Error searching for query '{query}': {e}")
                results.append([])  # Return empty results on error
        
        return results
    
    def predict_single(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """
        Convenience method for single query prediction.
        
        Args:
            query (str): Single query string
            top_k (int): Number of top results to return
            
        Returns:
            List[Dict]: List of top_k results for the query
        """
        results = self.predict([query], top_k)
        return results[0] if results else []
    
    def cleanup(self):
        """Delete the index and clean up resources."""
        if self.es and self.es.indices.exists(index=self.index_name):
            self.es.indices.delete(index=self.index_name)
            print(f"Deleted index '{self.index_name}'")

    def extract_features(self) -> Optional[Any]:
        """
        Extract features using RankingSHAP from the indexed documents.
        
        Returns:
            features: Extracted features or None if extraction fails
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before extracting features. Call .fit() first.")
        
        try:
            from rankingSHAP_test_class_cutomized import extract_es_features_optimized
            features = extract_es_features_optimized(self.es, self.index_name, field=self.field_name)
            return features
        except ImportError as e:
            print(f"Import error: {e}")
            print("Please check the path to your RankingSHAP module")
            return None
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None

# # Sample documents
# documents = [
#     "The stock market saw gains today in the finance sector.",
#     "Technology advances are happening at a rapid pace.",
#     "Healthy eating and exercise improve overall health.",
#     "Weather forecasts predict rain this week.",
#     "Financial experts recommend saving early for retirement.",
#     "Machine learning models require large datasets for training.",
#     "The weather today is sunny and warm.",
#     "Exercise and diet are key to maintaining good health."
# ]
documents = synthetic_documents.copy()

# Create and fit the model
model = ElasticsearchSparseSearchModel()
# Fit the model
print("Fitting model with documents...")
model.fit(documents)
features = model.extract_features()
print(features)
print(len(features))




Fitting model with documents...
Connected to Elasticsearch at http://localhost:9200
Created index 'sparse_search_e63693e0'
Indexed 100 documents
['a', 'accelerate', 'accuracy', 'achieve', 'advances', 'ai', 'amid', 'an', 'analysts', 'and', 'are', 'areas', 'at', 'automation', 'awareness', 'balanced', 'banking', 'becoming', 'been', 'bond', 'breakthroughs', 'chronic', 'coastal', 'complex', 'computational', 'computing', 'continue', 'data', 'datasets', 'deep', 'demands', 'depends', 'diet', 'diseases', 'early', 'eating', 'economic', 'enhance', 'exercise', 'expect', 'expected', 'experts', 'finance', 'financial', 'fine', 'fitness', 'for', 'forecasts', 'gains', 'good', 'greatly', 'happening', 'have', 'health', 'healthy', 'heatwave', 'heavily', 'heavy', 'high', 'hit', 'hydration', 'important', 'improve', 'in', 'increasingly', 'industries', 'industry', 'innovating', 'interest', 'investors', 'is', 'issued', 'key', 'labeled', 'large', 'learning', 'losses', 'machine', 'maintaining', 'market', 'mental

In [260]:
model.predict_single("financial market trends", top_k=3)

{'_index': 'sparse_search_e63693e0', '_id': '38', '_score': 2.5143557, '_source': {'content': 'Financial experts recommend saving early for retirement.'}}
{'_index': 'sparse_search_e63693e0', '_id': '47', '_score': 2.5143557, '_source': {'content': 'Financial experts recommend saving early for retirement.'}}
{'_index': 'sparse_search_e63693e0', '_id': '80', '_score': 2.5143557, '_source': {'content': 'Financial experts recommend saving early for retirement.'}}


[{'doc_id': '38',
  'score': 2.5143557,
  'content': 'Financial experts recommend saving early for retirement.'},
 {'doc_id': '47',
  'score': 2.5143557,
  'content': 'Financial experts recommend saving early for retirement.'},
 {'doc_id': '80',
  'score': 2.5143557,
  'content': 'Financial experts recommend saving early for retirement.'}]

In [204]:
from utils.pre_filtering import vocab_prune_by_global_tfidf, normalize_features

In [207]:


features = vocab_prune_by_global_tfidf(documents, keep_top_k=100, ngram_range=(1,1))
print("before stemming/lemmatization:", features)
print("number of features before stemming/lemmatization:", len(features))
features = normalize_features(features)
print("after stemming/lemmatization:", features)
print("number of features after stemming/lemmatization:", len(features))
#     features = {_simple_stem(t) for t in features}



before stemming/lemmatization: {'datasets', 'transforming', 'performance', 'physical', 'depends', 'healthy', 'computational', 'models', 'maintaining', 'soon', 'unprecedented', 'meteorologists', 'snowfall', 'areas', 'coastal', 'eating', 'demands', 'saw', 'complex', 'robotics', 'retirement', 'analysts', 'tech', 'role', 'early', 'bond', 'require', 'regular', 'sector', 'startups', 'rely', 'computing', 'shows', 'predict', 'region', 'chronic', 'rates', 'week', 'problems', 'reduces', 'southern', 'nutrition', 'diet', 'expected', 'showed', 'ai', 'learning', 'exercise', 'quantum', 'uncertainty', 'sunny', 'market', 'promise', 'steady', 'economic', 'today', 'pace', 'overall', 'optimism', 'rapidly', 'financial', 'balanced', 'key', 'awareness', 'data', 'saving', 'networks', 'optimal', 'deep', 'enhance', 'weather', 'preprocessing', 'rain', 'remain', 'solving', 'resources', 'temperatures', 'new', 'diseases', 'storm', 'slight', 'training', 'remained', 'warnings', 'health', 'breakthroughs', 'continue', 

80


In [168]:
# from typing import List, Tuple, Dict
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer


# def es_analyze_texts(es, index_name: str, texts: List[str], analyzer: str = "english") -> List[List[str]]:
#     toks: List[List[str]] = []
#     for t in texts:
#         try:
#             resp = es.indices.analyze(index=index_name, analyzer=analyzer, text=t)
#         except TypeError:
#             # fallback for older clients
#             resp = es.indices.analyze(index=index_name, body={"analyzer": analyzer, "text": t})
#         toks.append([tok["token"] for tok in resp.get("tokens", [])])
#     return toks

# def filter_features_by_idf_stemmed(
#     es, index_name: str, documents: List[str], stemmed_features: List[str],
#     top_k: int = None, min_idf: float = None, analyzer: str = "english"
# ) -> Tuple[List[str], Dict[str, float], TfidfVectorizer]:
#     tokenized_docs = es_analyze_texts(es, index_name, documents, analyzer=analyzer)
#     vec = TfidfVectorizer(
#         vocabulary=stemmed_features,
#         analyzer=lambda x: x,   # pre-tokenized stems
#         lowercase=False,
#         use_idf=True,
#         smooth_idf=True
#     )
#     X = vec.fit_transform(tokenized_docs)
#     vocab = vec.get_feature_names_out()
#     idf = vec.idf_

#     # drop stems that never occur in the corpus
#     df = (X > 0).sum(axis=0).A1
#     present_mask = df > 0
#     vocab_present = [vocab[i] for i, p in enumerate(present_mask) if p]
#     idf_present = [idf[i] for i, p in enumerate(present_mask) if p]
#     idf_map = dict(zip(vocab_present, idf_present))

#     if min_idf is not None:
#         kept = [t for t in vocab_present if idf_map[t] >= float(min_idf)]
#     else:
#         order = np.argsort(idf_present)[::-1]
#         kept = [vocab_present[i] for i in order[:top_k]] if top_k else list(vocab_present)
#     return kept, idf_map, vec
# # ...existing code...

# def select_query_stems_by_tfidf_es(
#     es, index_name: str, query: str, vec: TfidfVectorizer, top_k: int = None,
#     min_tfidf: float = None, analyzer: str = "english"
# ):
#     q_tokens = es_analyze_texts(es, index_name, [query], analyzer=analyzer)[0]
#     q_vec = vec.transform([q_tokens]).toarray()[0]
#     vocab = vec.get_feature_names_out()
#     if min_tfidf is not None:
#         kept = [vocab[i] for i, s in enumerate(q_vec) if s >= float(min_tfidf)]
#     else:
#         order = np.argsort(q_vec)[::-1]
#         kept = [vocab[i] for i in order[:top_k]] if top_k else [vocab[i] for i, s in enumerate(q_vec) if s > 0]
#     return kept

# # After you have: model.fit(documents) and features = model.extract_features()
# filtered_features, idf_map, vec = filter_features_by_idf_stemmed(
#     model.es, model.index_name, documents, features, top_k=50
# )
# print(f"Kept {len(filtered_features)} / {len(features)} stems")

# # # Optional per-query refinement:
# # query_kept = select_query_stems_by_tfidf_es(model.es, model.index_name, query, vec, top_k=50)
# # final_features = [t for t in filtered_features if t in set(query_kept)]

# # # Use the reduced feature set for RankingSHAP
# # features = final_features  # or filtered_features
# # background_data_new = np.full((1, len(features)), '<unk>', dtype=object)

In [169]:
# filtered_features

In [ ]:
from utils.get_explanations import calculate_all_query_explanations
from utils.helper_functions import get_data
import lightgbm
import numpy as np
from scipy.stats import kendalltau
from utils.background_data import BackgroundData
from approaches.ranking_shap import RankingShap
from approaches.ranking_lime import RankingLIME
from approaches.greedy_listwise import GreedyListwise
from approaches.pointwise_lime import AggregatedLime
from approaches.pointwise_shap import AggregatedShap
from approaches.random_explainer import RandomExplainer
from pathlib import Path


explanation_size = 1000
# methods_options = ["tf-idf", "bm25-idf", "intersection"]
# filter_method = methods_options[2]
# print("topk:", topk)
# print("Selected filter method:", filter_method)

# if filter_method == "tf-idf":
#     features = list(topk_vocab_tf_idf)
# if filter_method == "bm25-idf":
#     features = list(topk_vocab_BM25_idf)
# if filter_method == "intersection":
#     features = list(target_vocab)


num_features = len(features)
print(num_features)
background_data_new = np.full((1, num_features), '<unk>', dtype=object)
background_data_new

topk: 80
Selected filter method: intersection
56


array([['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']],
      dtype=object)

In [171]:
documents

['Financial experts recommend saving early for retirement.',
 'New breakthroughs in AI are transforming industries rapidly.',
 'Regular exercise and balanced nutrition enhance physical fitness.',
 'Exercise and diet are key to maintaining good health.',
 'Training neural networks demands substantial computational resources.',
 'The stock market saw gains today in the finance sector.',
 'The weather today is sunny and warm.',
 'Model performance depends greatly on data preprocessing.',
 'Maintaining a healthy diet reduces risk of chronic diseases.',
 'Deep learning models rely heavily on high-quality labeled data.',
 'Weather forecasts predict rain this week.',
 'Quantum computing shows promise for solving complex problems.',
 'Investors showed optimism in the banking industry this morning.',
 'Advances in robotics and automation continue to accelerate.',
 'Healthy eating and exercise improve overall health.',
 'Storm warnings have been issued for coastal areas.',
 'Tech startups are in

In [172]:
# _WORD = r"[A-Za-z]+(?:[-'][A-Za-z]+)*"          # handles hyphenated words & apostrophes: policy-maker, bank's
# _PLACEHOLDER = r"__PHRASE_\d+__"
# _PUNCT = r"[^\w\s]"                             # any single non-word, non-space (.,;:!?()[]{}"”’ etc.)
# _NUMBER = r"\d+(?:[.,]\d+)*%?|\$\d+(?:[.,]\d+)*"
# TOKEN_RX = re.compile(fr"{_PLACEHOLDER}|{_WORD}|{_PUNCT}|{_NUMBER}")

# def tokenize(text: str):
#     # Returns a list of tokens: words/placeholders/punctuation
#     return TOKEN_RX.findall(text)

# def detokenize(tokens):
#     out = []
#     for t in tokens:
#         if not out:
#             out.append(t)
#             continue
#         # If current token is punctuation, attach to previous without space
#         if re.fullmatch(_PUNCT, t):
#             out[-1] += t
#         else:
#             # otherwise add a space then the token
#             out.append(" " + t)
#     return "".join(out)

In [173]:
# detokenize(tokenize("AI models often require fine-tuning to achieve optimal accuracy. 123!"))    

In [266]:
len(documents)

27

In [273]:
documents = list(set(documents))

In [279]:
documents[1]

'New breakthroughs in AI are transforming industries rapidly.'

In [274]:
import sparse_search_bm25 as sparse_search_bm25
import importlib
importlib.reload(sparse_search_bm25)
import json

from sparse_search_bm25 import BM25SearchEngine, demo_bm25_search

# documents = list(set(documents))
print(len(documents))
query = "What is new in stock market?"
preds = demo_bm25_search(query,
                 documents)

27
Indexed 27 documents


In [275]:
len(preds)

27

In [276]:
preds[:5]

[{'_score': 5.0204769558105395,
  '_source': 'The stock market saw gains today in the finance sector.',
  '_index': 5},
 {'_score': 4.801017358713587,
  '_source': 'The stock market saw slight losses today in the tech sector.',
  '_index': 24},
 {'_score': 4.113410350132243,
  '_source': 'New breakthroughs in AI are transforming industries rapidly.',
  '_index': 1},
 {'_score': 2.0719043313682475,
  '_source': 'The weather today is sunny and warm.',
  '_index': 6},
 {'_score': 2.0719043313682475,
  '_source': 'Mental health awareness is becoming increasingly important.',
  '_index': 22}]

In [176]:
len(preds)

27

In [177]:




# import approaches.ranking_shap as ranking_shap_module
import importlib
from approaches import ranking_shap_decoupled_test_v3

importlib.reload(ranking_shap_decoupled_test_v3)
from approaches.ranking_shap_decoupled_test_v3 import RankingShapDecoupled

# Now use it with the module prefix
# approaches.ranking_shap_decoupled.your_function()

# from approaches.ranking_shap import RankingShap

import rbo
safe_attributions_to = '/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/test_results.csv'


# rank_similarity_coefficient = lambda x, y: rbo.RankingSimilarity(x, y)
rank_similarity_coefficient = lambda x, y: kendalltau(x, y)[0]
# Define all the explainers
ranking_shap_explainer = RankingShapDecoupled(
    permutation_sampler="kernel",
    background_data=background_data_new,
    original_model=demo_bm25_search,
    explanation_size=5, 
    rank_similarity_coefficient=rank_similarity_coefficient,
    nsample_permutations= 300
)

# feature_attribution = ranking_shap_explainer.get_query_explanation(
#     query_features=features, documents = documents, query = query
# )

# feature_attribution.safe_to_file(safe_attributions_to)
# features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
#     query_features=features, documents = documents, query_id=""
# )

# all_masks = ranking_shap_explainer.all_masks
# print(np.array(all_masks).shape)
# print(all_masks)


Using kernel weight mode: binom


In [178]:
masks, metadata = ranking_shap_explainer.generate_masks_only(features, 
                                           query_id= "123")

Generating masks for query: 123
Query features: ['slight', 'remained', 'transforming', 'performance', 'physical', 'promise', 'warnings', 'steady', 'economic', 'pace', 'quarter', 'overall', 'optimism', 'regular', 'uncertainty', 'soon', 'rapidly', 'recommend', 'startups', 'warm', 'rise', 'unprecedented', 'rely', 'meteorologists', 'snowfall', 'accelerate', 'shows', 'sleep', 'region', 'rates', 'saving', 'optimal', 'networks', 'problems', 'substantial', 'reduces', 'enhance', 'southern', 'nutrition', 'preprocessing', 'rain', 'play', 'showed', 'risk', 'remain', 'solving', 'resources', 'temperatures', 'robotics', 'new', 'quantum', 'retirement', 'role', 'sunny', 'early', 'storm']


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 12.25it/s]

Masks saved to: rankingshap_outputs/masks_123.npz
Metadata saved to: rankingshap_outputs/masks_123_metadata.json
Generated 301 masks


In [179]:
masks

array([['<keep>', '<keep>', '<keep>', ..., '<keep>', '<keep>', '<keep>'],
       ['<unk>', '<unk>', '<unk>', ..., '<unk>', '<unk>', '<unk>'],
       ['<keep>', '<keep>', '<keep>', ..., '<keep>', '<keep>', '<keep>'],
       ...,
       ['<keep>', '<keep>', '<keep>', ..., '<keep>', '<keep>', '<keep>'],
       ['<unk>', '<keep>', '<unk>', ..., '<unk>', '<unk>', '<unk>'],
       ['<keep>', '<unk>', '<keep>', ..., '<keep>', '<keep>', '<keep>']],
      dtype=object)

In [180]:
import os
# timestamp = "20251014_231705"
masks_file = os.path.join("rankingshap_outputs", f"masks_123.npz")

perturbed_results = ranking_shap_explainer.apply_masks_to_documents(masks_file,
                                                documents,
                                                query = query,
                                                output_filename=f"perturbed_data_123.npz")

Loaded 301 masks from rankingshap_outputs/masks_123.npz
Query features: ['slight', 'remained', 'transforming', 'performance', 'physical', 'promise', 'warnings', 'steady', 'economic', 'pace', 'quarter', 'overall', 'optimism', 'regular', 'uncertainty', 'soon', 'rapidly', 'recommend', 'startups', 'warm', 'rise', 'unprecedented', 'rely', 'meteorologists', 'snowfall', 'accelerate', 'shows', 'sleep', 'region', 'rates', 'saving', 'optimal', 'networks', 'problems', 'substantial', 'reduces', 'enhance', 'southern', 'nutrition', 'preprocessing', 'rain', 'play', 'showed', 'risk', 'remain', 'solving', 'resources', 'temperatures', 'robotics', 'new', 'quantum', 'retirement', 'role', 'sunny', 'early', 'storm']
Applying to 27 documents
Indexed 27 documents
Original ranking: [ 5 23  1  6 22 24 18 13 12 26  0  2  3  4  7  8  9 10 11 14 15 16 17 19
 20 21 25]
['Financial experts recommend saving early for retirement.', 'New breakthroughs in AI are transforming industries rapidly.', 'Regular exercise and b

In [181]:
len(perturbed_results)

301

In [182]:
num_masked = []
for mask_index in range(201):
    # mask = perturbed_results[mask_index][0]
    num_masked.append(len(perturbed_results[mask_index][2]['words_replaced']))

import matplotlib.pyplot as plt
plt.hist(num_masked, bins=20)

(array([37., 14., 13.,  4.,  7.,  8.,  4.,  3.,  4.,  6.,  4.,  3.,  5.,
         4.,  8.,  5.,  5., 10., 15., 42.]),
 array([ 0.  ,  2.75,  5.5 ,  8.25, 11.  , 13.75, 16.5 , 19.25, 22.  ,
        24.75, 27.5 , 30.25, 33.  , 35.75, 38.5 , 41.25, 44.  , 46.75,
        49.5 , 52.25, 55.  ]),
 <BarContainer object of 20 artists>)

In [183]:
perturbed_docs = perturbed_results[mask_index][1]
perturbed_docs

['Financial experts <unk> <unk> <unk> for <unk>.',
 '<unk> breakthroughs in AI are <unk> industries <unk>.',
 '<unk> exercise and balanced <unk> <unk> <unk> fitness.',
 'Exercise and diet are key to maintaining good health.',
 'Training neural <unk> demands <unk> computational <unk>.',
 'The stock market saw gains today in the finance sector.',
 'The weather today is <unk> and <unk>.',
 'Model <unk> depends greatly on data <unk>.',
 'Maintaining a healthy diet <unk> <unk> of chronic diseases.',
 'Deep learning models <unk> heavily on high-quality labeled data.',
 'Weather forecasts predict rain this week.',
 '<unk> computing <unk> <unk> for <unk> complex <unk>.',
 'Investors <unk> <unk> in the banking industry this morning.',
 'Advances in <unk> and automation continue to <unk>.',
 'Healthy eating and exercise improve <unk> health.',
 '<unk> <unk> have been issued for coastal areas.',
 'Tech <unk> are innovating at an <unk> <unk>.',
 'Financial analysts expect interest <unk> to <unk> n

In [184]:
perturbed_data_filepath = os.path.join("/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/rankingshap_outputs", 
                                       f"perturbed_data_123.npz")

data = np.load(perturbed_data_filepath, allow_pickle=True)


perturbed_data_filepath


'/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/rankingshap_outputs/perturbed_data_123.npz'

In [185]:
# for i in range(len(data["perturbed_data"])):
#     print(i, data["perturbed_data"][i]['perturbation_info']['similarity_score'])



In [186]:
# masks = []
# scores = []
# for item in data["perturbed_data"]:
#     masks.append(np.array(item['mask']))
#     scores.append(item['perturbation_info']['similarity_score'])

In [187]:
# perturbed_data_list = data["perturbed_data"]
# for item in perturbed_data_list:
#     masks.append(np.array(item['mask']))
#     scores.append(item['perturbation_info']['similarity_score'])

In [188]:
len(features)

56

In [189]:
len(features)

56

In [190]:
explanation_output = ranking_shap_explainer.calculate_shap_from_perturbed(
    perturbed_data_filepath,
    features,
    query_id ="123",
    documents=documents,
    query = query
)

Calculating SHAP values from: /Users/keqiaoli/Desktop/RankingSHAP/RankingShap/rankingshap_outputs/perturbed_data_123.npz
Loaded 301 precomputed mask-score pairs
Indexed 27 documents
Indexed 27 documents
Empty-coalition baseline (fnull) = 0.760684
Using kernel weight method: binom


In [191]:
explanation_output.safe_to_file('/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/test_results_intersection.csv')

Writing to  /Users/keqiaoli/Desktop/RankingSHAP/RankingShap/test_results_intersection.csv


In [ ]:
# pip install rank_bm25
from rank_bm25 import BM25Okapi

# 1. Prepare corpus
corpus = [
    "BM25 is a ranking function used by search engines.",
    "Elasticsearch uses BM25 as its default similarity algorithm.",
    "BM25Okapi is a Python implementation for text retrieval.",
    "This document talks about cooking pasta."
]

# 2. Tokenize (very basic split)
tokenized_corpus = [doc.lower().split() for doc in corpus]

# 3. Build BM25 model
bm25 = BM25Okapi(tokenized_corpus)

# 4. Query
query = "bm25 ranking algorithm".lower().split()

# 5. Get scores
scores = bm25.get_scores(query)

# 6. Sort and print top results
for idx in sorted(range(len(scores)), key=lambda i: scores[i], reverse=True):
    print(f"Doc {idx} | Score: {scores[idx]:.3f} | Text: {corpus[idx]}")


In [54]:
words_replaced = perturbed_results[1][2]['words_replaced']
print(words_replaced)

mask = perturbed_results[1][0]
mask

['technolog', 'gain', 'weather', 'maintain', 'stock', 'healthi', 'train', 'diet', 'happen', 'retir', 'improv', 'learn', 'saw', 'pace', 'good', 'model', 'eat', 'requir', 'health', 'expert', 'exercis', 'rapid', 'sector', 'overal', 'dataset', 'earli', 'market', 'week', 'financi', 'machin', 'larg', 'kei', 'rain', 'forecast', 'sunni', 'save', 'warm', 'recommend', 'todai']


array(['<keep>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<keep>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<keep>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
      dtype=object)

In [53]:
perturbed_docs = perturbed_results[1][1]
perturbed_docs

['The <unk> <unk> <unk> <unk> today in the <unk> <unk>.',
 '<unk> advances are <unk> at a <unk> <unk>.',
 '<unk> <unk> and exercise <unk> overall <unk>.',
 '<unk> <unk> predict <unk> this <unk>.',
 'Financial <unk> <unk> <unk> <unk> for <unk>.',
 '<unk> <unk> <unk> <unk> <unk> <unk> for <unk>.',
 'The <unk> today is <unk> and <unk>.',
 'Exercise and <unk> are key to <unk> <unk> <unk>.']

In [ ]:
perturbed_results[2][-1]['words_replaced']

In [44]:
perturbed_results[2][-1]['words_replaced']

['predict', 'advanc', 'financ']

In [6]:
# import approaches.ranking_shap as ranking_shap_module
# import importlib
# importlib.reload(ranking_shap_module)

# from approaches.ranking_shap import RankingShap




# rank_similarity_coefficient = lambda x, y: kendalltau(x, y)[0]

# # Define all the explainers
# ranking_shap_explainer = RankingShap(
#     permutation_sampler="kernel",
#     background_data=background_data_new,
#     original_model=model.predict,
#     explanation_size=3,
#     name="rankingshap",
#     rank_similarity_coefficient=rank_similarity_coefficient,
# )

# features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
#     query_features=features, query_id="12345"
# )


Indexed 8 documents

Query: 'What is new in stock market?'

1. Score: 4.3157

2. Score: 1.6758

3. Score: 0.0000


In [12]:
def rank_list(preds):
    """
    returns ndarray containing rank(i) for documents at position i
    """
    vector = np.array([pred['_score'] for pred in preds])
    temp = vector.argsort()[::-1]
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(1, len(vector) + 1)

    return ranks

In [13]:
rank_list(preds)

array([1, 2, 3])

  0%|          | 0/1 [00:00<?, ?it/s]

array================>: [['<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>']]
(1, 37)
==========> code goes here !!!!!
Query: What is new in stock market?
<function demo_bm25_search at 0x1780e2950>
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.0000

3. Score: 0.0000
Original rank: [1 3 2]

Mask 0: replaced_with_unk = set()
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 0: [1 3 2]
[1.0]
array================>: [['<keep>' '<unk>' '<unk>' ... '<unk>' '<unk>' '<unk>']
 ['<unk>' '<keep>' '<keep>' ... '<keep>' '<keep>' '<keep>']
 ['<unk>' '<keep>' '<unk>' ... '<unk>' '<unk>' '<unk

100%|██████████| 1/1 [00:00<00:00,  2.66it/s]


1. Score: 0.0000

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 668: [3 2 1]

Mask 669: replaced_with_unk = {'today', 'gains', 'health'}
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 669: [1 3 2]

Mask 670: replaced_with_unk = {'eating', 'and', 'health', 'finance', 'market', 'predict', 'stock', 'the', 'saw', 'this', 'today', 'experts', 'gains', 'happening', 'week', 'rain', 'saving', 'early', 'at', 'retirement', 'healthy', 'pace', 'forecasts', 'exercise', 'are', 'weather', 'a', 'improve', 'advances', 'rapid'}
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 0.4722

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 670: [1 3 2]

Mask 671: replaced_with_unk = {'sector', 'for', 'technology', 'financial', 'overall', 'recommend', 'in'}
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 0.9444

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 671: [1 3 2]

In [13]:
len(features), documents

(42,
 ['The stock market saw gains today in the finance sector.',
  'Technology advances are happening at a rapid pace.',
  'Healthy eating and exercise improve overall health.',
  'Weather forecasts predict rain this week.',
  'Financial experts recommend saving early for retirement.',
  'Machine learning models require large datasets for training.',
  'The weather today is sunny and warm.',
  'Exercise and diet are key to maintaining good health.'])

In [ ]:
import approaches.ranking_shap as ranking_shap_module
import importlib
importlib.reload(ranking_shap_module)

from approaches.ranking_shap import RankingShap
import rbo
safe_attributions_to = '/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/test_results.csv'


rank_similarity_coefficient = lambda x, y: rbo.RankingSimilarity(x, y)

# Define all the explainers
ranking_shap_explainer = RankingShap(
    permutation_sampler="kernel",
    background_data=background_data_new,
    original_model=demo_bm25_search,
    explanation_size=20,
    name="rankingshap",
    rank_similarity_coefficient=rank_similarity_coefficient,
    nsample_permutations=500
)

feature_attribution = ranking_shap_explainer.get_query_explanation(
    query_features=features, documents = documents, query = "query"
)

# feature_attribution.safe_to_file(safe_attributions_to)
# features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
#     query_features=features, documents = documents, query_id=""
# )

# all_masks = ranking_shap_explainer.all_masks
# print(np.array(all_masks).shape)
# print(all_masks)


100%|██████████| 1/1 [00:00<00:00,  7.59it/s]

array================>: [['<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>']]
(1, 42)
==========> code goes here !!!!!
Query: query
<function demo_bm25_search at 0x30057ee60>
Indexed 8 documents

Query: 'query'

1. Score: 0.0000

2. Score: 0.0000

3. Score: 0.0000
Original rank: [3 2 1]

Mask 0: replaced_with_unk = set()
Indexed 8 documents

Query: 'query'

1. Score: 0.0000

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 0: [3 2 1]
[1.0]
array================>: [['<keep>' '<unk>' '<unk>' ... '<unk>' '<unk>' '<unk>']
 ['<unk>' '<keep>' '<keep>' ... '<keep>' '<keep>' '<keep>']
 ['<unk>' '<unk>' '<keep>' ... '<unk>' '<keep>' '<unk>']
 ...
 ['<unk>' '<

In [13]:
ranking_shap_explainer.all_masks[1]

array(['<keep>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<keep>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<keep>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
      dtype=object)

In [15]:
masks_list = masks.tolist()
masks_list == ranking_shap_explainer.all_masks

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [18]:
masks.tolist()

[['<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>',
  '<keep>'],
 ['<keep>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<keep>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<keep>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>'],
 ['<unk>',
  '<keep>',
  '<ke

In [24]:
all_masks_rankingshape = [ele.tolist() for ele in ranking_shap_explainer.all_masks]

(all_masks_rankingshape == masks).all()

True

In [30]:
# import re

# def replace_words_in_sentences(documents, words_to_replace, unk_token="<unk>", case_sensitive=False):
#     if not words_to_replace:
#         return documents
    
#     # Sort by length to match longer words first (avoids subword matching confusion)
#     sorted_words = sorted(words_to_replace, key=len, reverse=True)
#     # Escape special characters
#     escaped_words = [re.escape(w) for w in sorted_words]
#     # Build regex pattern: \b matches word boundaries (including punctuation)
#     pattern_str = r'\b(' + '|'.join(escaped_words) + r')\b'
#     flags = 0 if case_sensitive else re.IGNORECASE
#     pattern = re.compile(pattern_str, flags)
#     # Replace, preserving surrounding punctuation
#     return [pattern.sub(unk_token, sent) for sent in documents]

# # Example setup
# query_features = ["a", "red", "cat", "stock", "today"]
# array = [
#     ["<keep>", "<unk>", "<keep>", "<keep>", "<unk>"],
#     ["<unk>", "<unk>", "<keep>", "<unk>", "<keep>"]
# ]
# documents = [
#     "today, the RED| cat jumps over the stock",
#     "a red cat in the stock market",
#     "today is a good day for the cat"
# ]

# # For each mask
# for mask_idx, mask in enumerate(array):
#     replaced_with_unk = set(q for q, a in zip(query_features, mask) if a == "<unk>")
#     print(f"\nMask {mask_idx}: replaced_with_unk = {replaced_with_unk}")
#     new_docs = replace_words_in_sentences(documents, replaced_with_unk)
#     for sent in new_docs:
#         print(sent)


In [32]:
documents

['The stock market saw gains today in the finance sector.',
 'Technology advances are happening at a rapid pace.',
 'Healthy eating and exercise improve overall health.',
 'Weather forecasts predict rain this week.',
 'Financial experts recommend saving early for retirement.',
 'Machine learning models require large datasets for training.',
 'The weather today is sunny and warm.',
 'Exercise and diet are key to maintaining good health.']

In [37]:
docuemnts

NameError: name 'docuemnts' is not defined

In [42]:
background_data_new

array([['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>']], dtype=object)

In [21]:
import approaches.ranking_shap as ranking_shap_module
import importlib
importlib.reload(ranking_shap_module)

from approaches.ranking_shap import RankingShap




rank_similarity_coefficient = lambda x, y: kendalltau(x, y)[0]

# Define all the explainers
ranking_shap_explainer = RankingShap(
    permutation_sampler="kernel",
    background_data=background_data_new,
    original_model=demo_bm25_search,
    explanation_size=20,
    name="rankingshap",
    rank_similarity_coefficient=rank_similarity_coefficient,
)

features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
    query_features=features, documents = documents, query_id=""
)


TypeError: RankingShap.get_query_explanation() got an unexpected keyword argument 'query_id'

In [30]:
import re
def replace_words_in_sentences(
    documents, 
    words_to_replace, 
    unk_token="<unk>", 
    case_sensitive=False
):
    """
    Replace query words in documents for perturbation analysis.
    Optimized for ranking/retrieval use cases.
    """
    # Quick return for empty cases
    if not documents or not words_to_replace:
        return documents
    
    # Ensure set for performance
    words_set = set(words_to_replace) if not isinstance(words_to_replace, set) else words_to_replace
    words_set = {w.strip() for w in words_set if w and w.strip()}
    
    if not words_set:
        return documents
    
    # Sort by length (longest first to avoid partial matches)
    sorted_words = sorted(words_set, key=len, reverse=True)
    
    # Escape and build pattern
    escaped_words = [re.escape(w) for w in sorted_words]
    
    # Pattern explanation:
    # (?<!\w) - not preceded by word character
    # (...) - capture group with words to replace
    # (?!\w) - not followed by word character
    # This handles: "word ", "word.", "word,", "-word-", etc.
    pattern_str = r'(?<!\w)(' + '|'.join(escaped_words) + r')(?!\w)'
    
    flags = 0 if case_sensitive else re.IGNORECASE
    
    try:
        pattern = re.compile(pattern_str, flags)
    except re.error as e:
        print(f"Regex compilation error: {e}")
        print(f"Pattern: {pattern_str[:100]}...")
        return documents  # Return original on error
    
    # Replace in all documents
    return [pattern.sub(unk_token, doc) if isinstance(doc, str) else doc 
            for doc in documents]



def test_replace_words_in_sentences():
    """Test suite for replace_words_in_sentences"""
    
    # Test 1: Basic replacement
    docs = ["information retrieval system"]
    words = {"retrieval"}
    result = replace_words_in_sentences(docs, words)
    assert result == ["information <unk> system"], f"Test 1 failed: {result}"
    
    # Test 2: Multiple words
    docs = ["effective information retrieval"]
    words = {"information", "retrieval"}
    result = replace_words_in_sentences(docs, words)
    assert result == ["effective <unk> <unk>"], f"Test 2 failed: {result}"
    
    # Test 3: Case insensitive (default)
    docs = ["Information Retrieval System"]
    words = {"information", "retrieval"}
    result = replace_words_in_sentences(docs, words, case_sensitive=False)
    assert result == ["<unk> <unk> System"], f"Test 3 failed: {result}"
    
    # Test 4: With punctuation
    docs = ["information, retrieval, and search."]
    words = {"retrieval", "search"}
    result = replace_words_in_sentences(docs, words)
    assert result == ["information, <unk>, and <unk>."], f"Test 4 failed: {result}"
    
    # Test 5: Hyphenated text
    docs = ["state-of-the-art retrieval"]
    words = {"of", "the", "retrieval"}
    result = replace_words_in_sentences(docs, words)
    # Note: This tests whether hyphens are handled correctly
    print(f"Test 5 result: {result}")
    
    # Test 6: Subword protection
    docs = ["information informative"]
    words = {"information"}
    result = replace_words_in_sentences(docs, words)
    assert result == ["<unk> informative"], f"Test 6 failed: {result}"
    
    # Test 7: Empty words
    docs = ["information retrieval"]
    words = set()
    result = replace_words_in_sentences(docs, words)
    assert result == docs, f"Test 7 failed: {result}"
    
    print("✅ All tests passed!")

# Run tests
test_replace_words_in_sentences()

Test 5 result: ['state-<unk>-<unk>-art <unk>']
✅ All tests passed!


1. The stock market saw gains today in the finance sector.
2. The bond market remained steady amid economic uncertainty.
3. The stock market saw slight losses today in the tech sector.
4. Weather forecasts predict rain this week.
5. Regular exercise and balanced nutrition enhance physical fitness.
6. Tech startups are innovating at an unprecedented pace.
7. Tech startups are innovating at an unprecedented pace.
8. New breakthroughs in AI are transforming industries rapidly.
9. Weather forecasts predict rain this week.
10. The bond market remained steady amid economic uncertainty.

Total records generated: 100


In [43]:
synthetic_documents

['The stock market saw gains today in the finance sector.',
 'The bond market remained steady amid economic uncertainty.',
 'The stock market saw slight losses today in the tech sector.',
 'Weather forecasts predict rain this week.',
 'Regular exercise and balanced nutrition enhance physical fitness.',
 'Tech startups are innovating at an unprecedented pace.',
 'Tech startups are innovating at an unprecedented pace.',
 'New breakthroughs in AI are transforming industries rapidly.',
 'Weather forecasts predict rain this week.',
 'The bond market remained steady amid economic uncertainty.',
 'Technology advances are happening at a rapid pace.',
 'Weather forecasts predict rain this week.',
 'AI models often require fine-tuning to achieve optimal accuracy.',
 'Financial analysts expect interest rates to rise next quarter.',
 'Model performance depends greatly on data preprocessing.',
 'A heatwave is expected to hit the southern region soon.',
 'Investors showed optimism in the banking ind

In [5]:
import numpy as np
import pandas as pd
import shap
import pytest

from approaches.ranking_shap import RankingShap, new_model_predict_val

def dummy_rank_model(docs):
    # docs: array-like, shape (num_docs, num_features)
    # return one score per doc (any deterministic function is fine)
    docs = np.array(docs, dtype=float)
    return docs.sum(axis=1)

def test_kernel_explainer_includes_empty_and_full_masks():
    num_docs = 3
    num_features = 5

    # Background data for SHAP (values used for imputation when a feature is "off")
    # Keep it simple; values don't matter for this test.
    background_data = np.zeros((1, num_features))

    # A "query" is a list/array of document feature vectors
    rng = np.random.default_rng(0)
    query_features = rng.normal(size=(num_docs, num_features))

    # Build the RankingShap with KernelExplainer
    rs = RankingShap(
        permutation_sampler="kernel",
        background_data=background_data,
        original_model=dummy_rank_model,
        explanation_size=3,
        nsample_permutations="auto",
    )

    # Wrap new_model_predict_val to capture masks that SHAP generates
    seen_masks = set()  # each entry is a tuple of 0/1 for a single coalition

    def logging_new_model_predict(array, **kwargs):
        # array: shape (batch, num_features). NaN means "feature ON (kept)".
        for a in array:
            m = tuple(int(x) for x in pd.isna(a))  # 1 = "on/keep", 0 = "off/impute"
            seen_masks.add(m)
        return new_model_predict_val(array, **kwargs)

    # Install the logging wrapper
    rs.new_model_predict = logging_new_model_predict

    # Run one explanation (this triggers KernelExplainer to generate the design)
    _sel, _attr = rs.get_query_explanation(query_features)

    # Build the two extremes
    full_mask = tuple([1] * num_features)   # all features "on"
    empty_mask = tuple([0] * num_features)  # all features "off"

    # Assert both extremes were observed
    assert full_mask in seen_masks, f"Full (all-ones) mask not observed. Seen masks: {sorted(seen_masks)}"
    assert empty_mask in seen_masks, f"Empty (all-zeros) mask not observed. Seen masks: {sorted(seen_masks)}"

    # Optional: print for debugging
    print(f"Observed {len(seen_masks)} unique masks; includes empty={empty_mask in seen_masks}, full={full_mask in seen_masks}")

In [6]:
test_kernel_explainer_includes_empty_and_full_masks()

TypeError: RankingShap.get_query_explanation() missing 1 required positional argument: 'documents'

In [7]:
# pip install elasticsearch==8.*  (or 7.* if your cluster is 7.x)
from elasticsearch import Elasticsearch, helpers
from uuid import uuid4
import random

# -----------------------------
# 1) Generate 100 synthetic docs with IDs
# -----------------------------
base_docs = [
    "The stock market saw gains today in the finance sector.",
    "Technology advances are happening at a rapid pace.",
    "Healthy eating and exercise improve overall health.",
    "Weather forecasts predict rain this week.",
    "Financial experts recommend saving early for retirement.",
    "Machine learning models require large datasets for training.",
    "The weather today is sunny and warm.",
    "Exercise and diet are key to maintaining good health."
]

finance_variations = [
    "The stock market saw slight losses today in the tech sector.",
    "Investors showed optimism in the banking industry this morning.",
    "Financial analysts expect interest rates to rise next quarter.",
    "The bond market remained steady amid economic uncertainty."
]
tech_variations = [
    "New breakthroughs in AI are transforming industries rapidly.",
    "Advances in robotics and automation continue to accelerate.",
    "Quantum computing shows promise for solving complex problems.",
    "Tech startups are innovating at an unprecedented pace."
]
health_variations = [
    "Regular exercise and balanced nutrition enhance physical fitness.",
    "Maintaining a healthy diet reduces risk of chronic diseases.",
    "Sleep and hydration play a key role in wellness.",
    "Mental health awareness is becoming increasingly important."
]
weather_variations = [
    "Meteorologists predict heavy snowfall this weekend.",
    "A heatwave is expected to hit the southern region soon.",
    "Storm warnings have been issued for coastal areas.",
    "Temperatures are expected to remain mild throughout the week."
]
ml_variations = [
    "Deep learning models rely heavily on high-quality labeled data.",
    "AI models often require fine-tuning to achieve optimal accuracy.",
    "Model performance depends greatly on data preprocessing.",
    "Training neural networks demands substantial computational resources."
]

all_templates = (
    base_docs
    + finance_variations
    + tech_variations
    + health_variations
    + weather_variations
    + ml_variations
)

def synth_text():
    # light lexical variation
    t = random.choice(all_templates)
    addons = ["", "", "", " (report)", " update", " - analysis", " insights"]
    return t.replace("today", random.choice(["today", "this morning", "earlier"]))
    + random.choice(addons)

topics = {
    "finance": finance_variations + [base_docs[0], base_docs[4]],
    "tech": tech_variations,
    "health": health_variations + [base_docs[2], base_docs[7]],
    "weather": weather_variations + [base_docs[3], base_docs[6]],
    "ml": ml_variations + [base_docs[5]],
}

def infer_topic(text: str) -> str:
    for k, vals in topics.items():
        if any(v.split()[0] in text for v in vals):  # quick heuristic
            return k
    # fallback keywords
    text_l = text.lower()
    if "market" in text_l or "financial" in text_l or "investor" in text_l:
        return "finance"
    if "ai" in text_l or "tech" in text_l or "quantum" in text_l:
        return "tech"
    if "exercise" in text_l or "diet" in text_l or "health" in text_l:
        return "health"
    if "weather" in text_l or "forecast" in text_l or "storm" in text_l:
        return "weather"
    if "model" in text_l or "training" in text_l or "neural" in text_l:
        return "ml"
    return "other"

docs = []
for _ in range(100):
    content = synth_text()
    docs.append({
        "_id": str(uuid4()),           # stable unique ID you can subset on
        "content": content,
        "topic": infer_topic(content)  # optional metadata
    })



In [21]:
from elasticsearch import Elasticsearch, helpers
from typing import List, Dict, Any, Optional, Union
import uuid

class ElasticsearchSparseSearchModel:
    """
    A simple Elasticsearch sparse search model with scikit-learn-like interface.
    Uses BM25 scoring for sparse retrieval and supports subset-restricted search.
    """
    
    def __init__(self, es_host: str = "http://localhost:9200", index_name: Optional[str] = None):
        self.es_host = es_host
        self.index_name = index_name or f"sparse_search_{uuid.uuid4().hex[:8]}"
        self.es = None
        self.is_fitted = False
        self.field_name = "content"
        
    def _connect(self):
        if self.es is None:
            try:
                self.es = Elasticsearch(self.es_host)
                if not self.es.ping():
                    raise ConnectionError("Cannot connect to Elasticsearch")
                print(f"Connected to Elasticsearch at {self.es_host}")
            except Exception as e:
                raise ConnectionError(f"Failed to connect to Elasticsearch: {e}")
    
    def fit(self, 
            documents: List[str], 
            splits: Optional[List[Optional[str]]] = None,
            mask_indices: Optional[List[int]] = None
           ) -> 'ElasticsearchSparseSearchModel':
        """
        Index documents. Optionally tag each doc with a 'split' (e.g., 'train'/'test'/'oot').

        Args:
            documents: list of document texts
            splits: optional list the same length as documents, containing split names or None
        """
        self._connect()
        
        # Delete existing index if it exists
        if self.es.indices.exists(index=self.index_name):
            self.es.indices.delete(index=self.index_name)
            print(f"Deleted existing index '{self.index_name}'")
        
        # Create index with BM25 settings and extra fields for filtering
        mapping = {
            "settings": {
                "index": {
                    # BM25 is default; left implicit, but you could define a named similarity here.
                }
            },
            "mappings": {
                "properties": {
                    self.field_name: {"type": "text", "analyzer": "english"},
                    "doc_id": {"type": "keyword"},
                    "split": {"type": "keyword"},
                    "mask_index": {"type": "integer"}  # <- store your external metadata
                }
            }
        }
        
        try:
            self.es.indices.create(index=self.index_name, **mapping)
            print(f"Created index '{self.index_name}'")
        except Exception as e:
            raise RuntimeError(f"Failed to create index: {e}")
        
        if splits is not None and len(splits) != len(documents):
            raise ValueError("If provided, 'splits' must be the same length as 'documents'.")
        
        # Index documents
        actions = []
        for i, doc in enumerate(documents):
            _id = i  # stable numeric IDs for convenience
            src = {
                self.field_name: doc,
                "doc_id": str(_id)
            }
            if splits is not None:
                src["split"] = splits[i] if splits[i] is not None else None
            if mask_indices is not None:
                src["mask_index"] = mask_indices[i]   # <- persist external metadata
            actions.append({
                "_index": self.index_name,
                "_id": _id,
                "_source": src
            })
        
        try:
            helpers.bulk(self.es, actions)
            self.es.indices.refresh(index=self.index_name)
            print(f"Indexed {len(documents)} documents")
            self.is_fitted = True
        except Exception as e:
            raise RuntimeError(f"Failed to index documents: {e}")
        
        return self

    def _build_filter_clauses(
        self, 
        subset_ids: Optional[List[Union[int, str]]] = None, 
        subset_filters: Optional[Dict[str, Union[str, List[str]]]] = None
    ) -> List[Dict[str, Any]]:
        """
        Build ES bool.filter clauses from subset arguments.
        """
        filters: List[Dict[str, Any]] = []
        
        # Restrict by _id (fast path via 'ids' query)
        if subset_ids:
            # Convert all to strings because ES _id is string-typed internally
            values = [str(x) for x in subset_ids]
            filters.append({"ids": {"values": values}})
        
        # Arbitrary field-based filters (e.g., {"split": "train"} or {"doc_id": ["1","2"]})
        if subset_filters:
            for field, val in subset_filters.items():
                if isinstance(val, list):
                    filters.append({"terms": {field: val}})
                else:
                    filters.append({"term": {field: val}})
        
        return filters
    
    # 1) Change the signature of predict and predict_single
    def predict(
        self, 
        queries: List[str], 
        top_k: int = 5, 
        subset_ids: Optional[List[Union[int, str]]] = None,
        subset_filters: Optional[Dict[str, Union[str, List[str]]]] = None,
        return_all_within_subset: bool = False
    ) -> List[List[Dict[str, Any]]]:

        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before making predictions. Call .fit() first.")
        
        results: List[List[Dict[str, Any]]] = []
        filters = self._build_filter_clauses(subset_ids=subset_ids, subset_filters=subset_filters)

        # If returning all within subset, set size accordingly.
        # If subset_ids provided, use its length; otherwise use a safe upper bound.
        # Note: ES max_result_window default is 10000. For more, use search_after/scroll.
        if return_all_within_subset:
            if subset_ids:
                size_for_all = len(subset_ids)
            else:
                size_for_all = 10000  # safe cap if using field filters only
        else:
            size_for_all = top_k

        for query in queries:
            search_body: Dict[str, Any] = {
                "query": {
                    "bool": {
                        "must": {"match": {self.field_name: query}},
                        "filter": filters if filters else []
                    }
                },
                "size": size_for_all,
                "track_total_hits": True,
                "_source": [self.field_name, "doc_id", "split", "mask_index"]  # include mask_index
            }
            try:
                response = self.es.search(index=self.index_name, body=search_body)
                query_results = []
                for hit in response['hits']['hits']:
                    src = hit['_source']
                    query_results.append({
                        'doc_id': hit['_id'],
                        'score': hit['_score'],
                        'content': src.get(self.field_name, ""),
                        'split': src.get('split', None),
                        'doc_id_field': src.get('doc_id', None),
                        'mask_index': src.get('mask_index', None),  # expose mask_index
                    })
                results.append(query_results)
            except Exception as e:
                print(f"Error searching for query '{query}': {e}")
                results.append([])
        return results

    def predict_single(
        self, 
        query: str, 
        top_k: int = 5, 
        subset_ids: Optional[List[Union[int, str]]] = None,
        subset_filters: Optional[Dict[str, Union[str, List[str]]]] = None,
        return_all_within_subset: bool = False
    ) -> List[Dict[str, Any]]:
        res = self.predict(
            [query], 
            top_k=top_k, 
            subset_ids=subset_ids, 
            subset_filters=subset_filters,
            return_all_within_subset=return_all_within_subset
        )
        return res[0] if res else []

    def predict_by_mask(
        self, query: str, mask_index: int, return_all_within_subset: bool = True, top_k: int = 5
    ):
        return self.predict_single(
            query,
            top_k=top_k,
            subset_filters={"mask_index": mask_index},
            return_all_within_subset=return_all_within_subset
        )
        

# # ----------------- Example usage -----------------



import random
from typing import List
# from your_module import ElasticsearchSparseSearchModel  # use your actual import

# ---------- 1) Build 100 synthetic docs ----------
base_docs = [
    "The stock market saw gains today in the finance sector.",
    "Technology advances are happening at a rapid pace.",
    "Healthy eating and exercise improve overall health.",
    "Weather forecasts predict rain this week.",
    "Financial experts recommend saving early for retirement.",
    "Machine learning models require large datasets for training.",
    "The weather today is sunny and warm.",
    "Exercise and diet are key to maintaining good health."
]

finance = [
    "Investors showed optimism in the banking industry this morning.",
    "Financial analysts expect interest rates to rise next quarter.",
    "The bond market remained steady amid economic uncertainty.",
    "Banks reported mixed earnings in the latest quarter."
]
tech = [
    "New breakthroughs in AI are transforming industries rapidly.",
    "Advances in robotics and automation continue to accelerate.",
    "Quantum computing shows promise for solving complex problems.",
    "Tech startups are innovating at an unprecedented pace."
]
health = [
    "Regular exercise and balanced nutrition enhance physical fitness.",
    "Maintaining a healthy diet reduces risk of chronic diseases.",
    "Sleep and hydration play a key role in wellness.",
    "Mental health awareness is becoming increasingly important."
]
weather = [
    "Meteorologists predict heavy snowfall this weekend.",
    "A heatwave is expected to hit the southern region soon.",
    "Storm warnings have been issued for coastal areas.",
    "Temperatures are expected to remain mild throughout the week."
]
ml = [
    "Deep learning models rely heavily on high-quality labeled data.",
    "AI models often require fine-tuning to achieve optimal accuracy.",
    "Model performance depends greatly on data preprocessing.",
    "Training neural networks demands substantial computational resources."
]

all_templates: List[str] = base_docs + finance + tech + health + weather + ml

def synth_text():
    # light lexical variation
    t = random.choice(all_templates)
    addons = ["", "", "", " (report)", " update", " - analysis", " insights"]
    return (
        t.replace("today", random.choice(["today", "this morning", "earlier"]))
        + random.choice(addons)
    )

documents = [synth_text() for _ in range(100)]

# Deterministic splits: 60 train, 20 test, 20 oot
splits = (["train"] * 60) + (["test"] * 20) + (["oot"] * 20)


documents = [synth_text() for _ in range(100)]
mask_indices = [i % 5 for i in range(100)]  # external metadata: 5 subsets

# for i in range(5):
#     print(f"Documents in mask index {i}:")
#     for doc, mask in zip(documents, mask_indices):
#         if mask == i:
#             print(f" - {doc}")

# -----------------------------------------------
# Index and search
# -----------------------------------------------
model = ElasticsearchSparseSearchModel()
model.fit(documents, mask_indices)

# Search only within mask_index = 3
subset_hits = model.predict_by_mask(query="Weather forecasts", mask_index=0, return_all_within_subset=True)

print(f"\nReturned {len(subset_hits)} hits from subset mask_index=3\n")
for h in subset_hits[:10]:
    print(f"mask_index={h['mask_index']}, score={h['score']:.3f}, text={h['content']}")




Connected to Elasticsearch at http://localhost:9200
Created index 'sparse_search_9bbec3fa'
Indexed 100 documents

Returned 0 hits from subset mask_index=3



In [225]:
def _compute_shap_from_masks_and_scores(
    self, 
    masks: np.ndarray, 
    scores: np.ndarray, 
    num_features: int,
    fnull: float | None = None,
    fx: float | None = None,
) -> np.ndarray:
    """
    KernelSHAP-compatible solve from precomputed masks and scores.
    Mirrors the SHAP KernelExplainer single-output solve:
    - compute kernel weights for interior coalitions
    - optional L1 feature selection (LARS)
    - eliminate one variable using efficiency constraint
    - weighted least squares with solve -> lstsq fallback
    """
    import numpy as np

    # 0) Deduplicate identical masks; average scores; keep replicate counts as weights
    masks, scores, repeat_counts = self._deduplicate_masks_and_scores(masks, scores)

    # 1) Binary mask matrix and coalition sizes
    maskMatrix = self._convert_masks_to_binary(masks, num_features).astype(float)  # (n, M)
    M = int(num_features)
    s = maskMatrix.sum(axis=1).astype(int)

    # 2) Filter non-finite scores early
    finite = np.isfinite(scores)
    if not np.all(finite):
        maskMatrix = maskMatrix[finite]
        s = s[finite]
        repeat_counts = repeat_counts[finite]
        scores = scores[finite]

    # 3) Infer fnull (s=0) and fx (s=M) from boundary coalitions when present
    idx_empty = np.where(s == 0)[0]
    idx_full = np.where(s == M)[0]
    if fnull is None and idx_empty.size > 0:
        fnull = np.average(scores[idx_empty], weights=repeat_counts[idx_empty])
    if fx is None and idx_full.size > 0:
        fx = np.average(scores[idx_full], weights=repeat_counts[idx_full])
    if fx is None:
        fx = 1.0  # identical ranking similarity
    if fnull is None:
        # Last resort: interior-weighted average as a baseline
        interior_tmp = (s > 0) & (s < M)
        if not np.any(interior_tmp):
            raise ValueError("No interior coalitions available to estimate fnull.")
        w_tmp = self._compute_shapley_kernel_weights(s[interior_tmp], M) * repeat_counts[interior_tmp]
        fnull = np.average(scores[interior_tmp], weights=w_tmp)

    # 4) Adjusted targets (identity link)
    eyAdj = scores - fnull         # eyAdj = link(ey) - link(fnull)
    delta = fx - fnull             # link(fx) - link(fnull)

    # 5) Keep only interior coalitions for the regression
    interior = (s > 0) & (s < M)
    if not np.any(interior):
        raise ValueError("Insufficient interior coalitions (need interior 0 < |S| < M).")
    maskMatrix = maskMatrix[interior]
    eyAdj = eyAdj[interior]
    s = s[interior]
    repeat_counts = repeat_counts[interior]

    # 6) SHAP kernel weights times repeat counts (equivalent to row replication)
    kernelWeights = self._compute_shapley_kernel_weights(s, M)
    kernelWeights = kernelWeights * repeat_counts

    # 7) Optional L1 feature selection (SHAP-style) using augmented, row-weighted system
    nonzero_inds = np.arange(M)
    if hasattr(self, 'l1_reg') and isinstance(self.l1_reg, str) and self.l1_reg.startswith("num_features("):
        try:
            from sklearn.linear_model import lars_path
            r = int(self.l1_reg[len("num_features(") : -1])

            # SHAP augments the system to include the constraint
            w_aug = np.hstack((kernelWeights * (M - s), kernelWeights * s))
            w_sqrt_aug = np.sqrt(w_aug)

            eyAdj_aug = np.hstack((eyAdj, eyAdj - delta))
            eyAdj_aug *= w_sqrt_aug

            mask_aug = np.vstack((maskMatrix, maskMatrix - 1.0))
            mask_aug = (w_sqrt_aug[:, None] * mask_aug)

            # LARS path; collect active indices up to r
            _, active, _ = lars_path(mask_aug, eyAdj_aug, max_iter=r)
            nonzero_inds = np.array(active, dtype=int)
            if nonzero_inds.size == 0:
                nonzero_inds = np.arange(M)
            elif nonzero_inds.size < 2:
                nonzero_inds = np.concatenate([nonzero_inds, [np.setdiff1d(np.arange(M), nonzero_inds)[0]]])
        except Exception as e:
            print(f"NOTE: L1 skipped: {e}")
            nonzero_inds = np.arange(M)

    # 8) Eliminate one variable with efficiency constraint (exactly as in SHAP)
    if nonzero_inds.size > 1:
        col_var = maskMatrix[:, nonzero_inds].var(axis=0)
        pivot_idx = np.argmax(col_var)
        pivot = nonzero_inds[pivot_idx]
        others = np.delete(nonzero_inds, pivot_idx)
    else:
        pivot = nonzero_inds[0]
        others = np.array([], dtype=int)
    
    eyAdj2 = eyAdj - maskMatrix[:, pivot] * delta
    etmp = (maskMatrix[:, others].T - maskMatrix[:, pivot]).T
    # 9) Weighted least squares solve with SHAP’s fallback strategy
    y = np.asarray(eyAdj2)
    X = etmp
    W = kernelWeights

    # Fast path: normal equations; fallback to lstsq on row-weighted system
    try:
        # WX = W[:, None] * X
        # w_sol = np.linalg.solve(X.T @ WX, WX.T @ y)
        WX = W[:, None] * X
        XtWX = X.T @ WX
        XtWy = WX.T @ y
        alpha = 1e-6 * (np.linalg.norm(XtWX, ord=2) + 1e-12)
        w_sol = np.linalg.solve(XtWX + alpha * np.eye(XtWX.shape[0]), XtWy)
    except np.linalg.LinAlgError:
        import warnings
        warnings.warn(
            "Linear regression equation is singular, a least squares solution is used instead.\n"
            "To avoid this situation and get a regular matrix do one of the following:\n"
            "1) turn up the number of samples,\n"
            "2) turn up the L1 regularization with num_features(N) where N is less than the number of samples,\n"
            "3) group features together to reduce the number of inputs that need to be explained."
        )
        sqrt_W = np.sqrt(W)
        w_sol = np.linalg.lstsq(sqrt_W[:, None] * X, sqrt_W * y, rcond=None)[0]

    # 10) Reconstruct full phi and clean tiny numerical noise
    phi = np.zeros(M)
    phi[others] = w_sol
    phi[pivot] = delta - w_sol.sum()
    phi[np.isclose(phi, 0.0, atol=1e-10)] = 0.0

    # Sanity check for efficiency
    if not np.isclose(phi.sum(), delta, atol=1e-8):
        print(f"WARNING: sum(phi)={phi.sum():.8f} differs from delta={delta:.8f}")

    return phi





def _compute_shapley_kernel_weights(
    self,
    coalition_sizes,
    num_features: int,
    method: str = None
):
    """
    SHAP kernel weights for interior subset sizes:
        w(s) = (M-1) / [C(M,s) * s * (M-s)], for 0 < s < M

    method:
        - 'gammaln' (default): compute in log-space using gammaln for stability
        - 'binom': use scipy.special.binom (fast; fine for moderate M)
    """
    import numpy as np

    M = int(num_features)
    s = np.asarray(coalition_sizes, dtype=int)
    w = np.zeros_like(s, dtype=float)

    interior = (s > 0) & (s < M)
    if not np.any(interior):
        return w

    method = (method or getattr(self, "kernel_weight_mode", "gammaln")).lower()
    print("Using kernel weight method:", method)
    s_i = s[interior].astype(float)

    if method == "binom":
        try:
            from scipy.special import binom
            C = binom(M, s_i)
        except Exception:
            from math import comb as _comb
            C = np.array([float(_comb(M, int(k))) for k in s_i], dtype=float)
        denom = C * s_i * (M - s_i)
        w[interior] = (M - 1.0) / denom
    elif method == "gammaln":
        try:
            from scipy.special import gammaln
            # log C(M, s) = lgamma(M+1) - lgamma(s+1) - lgamma(M-s+1)
            logC = gammaln(M + 1.0) - gammaln(s_i + 1.0) - gammaln(M - s_i + 1.0)
            # log denom = logC + log s + log(M - s)
            log_denom = logC + np.log(s_i) + np.log(M - s_i)
            # log w = log(M-1) - log_denom
            log_w = np.log(M - 1.0) - log_denom
            w[interior] = np.exp(log_w)
        except Exception:
            # Fallback to binom path if SciPy missing
            from math import comb as _comb
            C = np.array([float(_comb(M, int(k))) for k in s_i], dtype=float)
            denom = C * s_i * (M - s_i)
            w[interior] = (M - 1.0) / denom
    else:
        raise ValueError("method must be 'gammaln' or 'binom'")

    return w










# def _weighted_least_squares_efficient(self, X, y, weights):
#     """
#     Efficient weighted least squares via row scaling, no intercept.
#     Solves: argmin_b || diag(sqrt(w)) (X b - y) ||_2
#     Returns coefficients b of shape (X.shape[1],)
#     """
#     X = np.asarray(X, dtype=float)
#     y = np.asarray(y, dtype=float)
#     w = np.asarray(weights, dtype=float)

#     sqrt_w = np.sqrt(w)
#     Xw = X * sqrt_w[:, np.newaxis]
#     yw = y * sqrt_w
    
#     # Solve using stable method
#     try:
#         # lstsq uses QR decomposition internally
#         coefficients, residuals, rank, singular_values = np.linalg.lstsq(
#             Xw, 
#             yw, 
#             rcond=None
#         )
        
#         # Check conditioning
#         if rank < X.shape[1]:
#             print(f"WARNING: Rank deficient matrix (rank={rank}/{X.shape[1]})")
            
#     except np.linalg.LinAlgError as e:
#         print(f"LinAlgError: {e}, using pseudoinverse")
#         coefficients = np.linalg.pinv(Xw) @ yw
    
#     return coefficients


def _deduplicate_masks_and_scores(self, masks, scores):
    """
    Collapse duplicate masks by averaging scores and increasing weights.
    """
    from collections import defaultdict
    
    # Group by mask
    mask_to_indices = defaultdict(list)
    for i, mask in enumerate(masks):
        mask_key = tuple(mask.flatten())
        mask_to_indices[mask_key].append(i)
    
    # Deduplicate
    unique_masks = []
    unique_scores = []
    repeat_counts = []
    
    for mask_key, indices in mask_to_indices.items():
        unique_masks.append(masks[indices[0]])
        # Average scores for duplicates
        unique_scores.append(np.mean([scores[i] for i in indices]))
        repeat_counts.append(len(indices))
    
    if len(unique_masks) < len(masks):
        print(f"Collapsed {len(masks)} masks to {len(unique_masks)} unique masks")
    
    return np.array(unique_masks), np.array(unique_scores), np.array(repeat_counts)                   

In [226]:
# Test with 3 features where you can compute SHAP analytically
def test_simple_case():
    M = 3
    # All possible coalitions
    all_masks = [
        ['<unk>', '<unk>', '<unk>'],  # empty
        ['<keep>', '<unk>', '<unk>'],
        ['<unk>', '<keep>', '<unk>'],
        ['<unk>', '<unk>', '<keep>'],
        ['<keep>', '<keep>', '<unk>'],
        ['<keep>', '<unk>', '<keep>'],
        ['<unk>', '<keep>', '<keep>'],
        ['<keep>', '<keep>', '<keep>'],  # full
    ]
    
    # Mock scores (design to have known SHAP values)
    scores = np.array([0.0, 0.3, 0.5, 0.2, 0.7, 0.6, 0.8, 1.0])
    
    
    # Run your method
    shap_vals = _compute_shap_from_masks_and_scores(
        np.array(all_masks), scores, M, fnull=0.0, fx=1.0
    )
    
    # Verify: sum should equal 1.0 (fx - fnull)
    assert np.isclose(shap_vals.sum(), 1.0)
    
    # Compare with shap.KernelExplainer on same data
    # ...

In [1]:
"""
Leverage Elasticsearch BM25 as tokenizer and select top-k features by BM25 IDF scores.
"""

import math
from typing import Dict, List, Tuple, Optional, Any
from elasticsearch import Elasticsearch


# =============================================================================
# Core Function: BM25 Tokenization + Top-K Feature Selection
# =============================================================================

def select_top_k_features_by_bm25_idf(
    es,
    index: str,
    field: str,
    query_text: str,
    top_k: int = 10,
    min_idf: Optional[float] = None,
) -> Tuple[List[str], Dict[str, float], Dict[str, Any]]:
    """
    Use ES BM25 as tokenizer and select top-k features by BM25 IDF score.
    
    Workflow:
    1. ES analyzes query_text using the field's analyzer (tokenization, stemming, etc.)
    2. ES computes BM25 IDF scores: log(1 + (N - df + 0.5)/(df + 0.5))
    3. Tokens are ranked by IDF score and top-k are selected
    
    Args:
        es: Elasticsearch client
        index: Index name
        field: Field name (uses this field's analyzer)
        query_text: Text to tokenize and analyze
        top_k: Number of top features to return
        min_idf: Optional minimum IDF threshold
    
    Returns:
        Tuple of (top_k_tokens, idf_scores, metadata)
    
    Example:
        >>> es = Elasticsearch(['http://localhost:9200'])
        >>> tokens, scores, meta = select_top_k_features_by_bm25_idf(
        ...     es, "articles", "content",
        ...     "machine learning and artificial intelligence",
        ...     top_k=3
        ... )
        >>> print(tokens)  # ['intelligence', 'artificial', 'machine']
        >>> print(scores)  # {'intelligence': 4.2, 'artificial': 3.8, ...}
    """
    
    # Step 1: Get BM25 IDF scores from ES using termvectors
    # This automatically tokenizes using the field's analyzer
    try:
        resp = es.termvectors(
            index=index,
            doc={field: query_text or ""},
            fields=[field],
            term_statistics=True,
            positions=False,
            offsets=False,
            payloads=False,
        )
    except TypeError:
        # ES 7.x compatibility
        resp = es.termvectors(
            index=index,
            body={
                "doc": {field: query_text or ""},
                "fields": [field],
                "term_statistics": True,
                "positions": False,
                "offsets": False,
                "payloads": False,
            },
        )
    
    # Step 2: Extract term statistics and compute BM25 IDF
    tv = (resp or {}).get("term_vectors", {}).get(field, {}) or {}
    N = int(tv.get("field_statistics", {}).get("doc_count", 0))
    terms = tv.get("terms", {}) or {}
    
    idf_scores: Dict[str, float] = {}
    for term, stats in terms.items():
        df = int(stats.get("doc_freq", 0))
        if N > 0 and df > 0:
            # BM25 IDF formula
            idf = math.log1p((N - df + 0.5) / (df + 0.5))
            idf_scores[term] = idf
    
    # Step 3: Filter by minimum IDF if specified
    if min_idf is not None:
        idf_scores = {t: s for t, s in idf_scores.items() if s >= min_idf}
    
    # Step 4: Sort by IDF score (descending) and select top-k
    sorted_terms = sorted(idf_scores.items(), key=lambda x: x[1], reverse=True)
    top_k_terms = [term for term, score in sorted_terms[:top_k]]
    
    metadata = {
        "N": N,
        "total_unique_terms": len(terms),
        "top_k": top_k,
        "mode": "bm25_tokenization"
    }
    
    return top_k_terms, idf_scores, metadata



In [ ]:

# =============================================================================
# Example 1: Basic Usage - Top-K Feature Selection
# =============================================================================

def example_basic_top_k_selection():
    """Select top-k most distinctive terms from a query."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    query_text = "deep learning neural networks for natural language processing"
    
    # Get top 5 features by BM25 IDF
    top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
        es=es,
        index="research_papers",
        field="abstract",
        query_text=query_text,
        top_k=5
    )
    
    print(f"Original query: {query_text}")
    print(f"\nTop {meta['top_k']} features by BM25 IDF:")
    for i, feature in enumerate(top_features, 1):
        print(f"  {i}. {feature:20s} (IDF: {idf_scores[feature]:.4f})")
    
    print(f"\nTotal unique terms analyzed: {meta['total_unique_terms']}")
    print(f"Total documents in index: {meta['N']}")


# =============================================================================
# Example 2: Feature Selection with Minimum IDF Threshold
# =============================================================================

def example_with_min_idf():
    """Filter out common words using minimum IDF threshold."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    query_text = "how to implement a machine learning model in python"
    
    # Only keep terms with IDF >= 2.0 (filters common words)
    top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
        es=es,
        index="tutorials",
        field="content",
        query_text=query_text,
        top_k=10,
        min_idf=2.0  # Threshold to remove stopwords and common terms
    )
    
    print(f"Query: {query_text}")
    print(f"\nFeatures with IDF >= 2.0:")
    for feature in top_features:
        print(f"  • {feature}: {idf_scores[feature]:.4f}")


# =============================================================================
# Example 3: Build Enhanced BM25 Search Query
# =============================================================================

def example_enhanced_bm25_search():
    """Use top-k features to build an enhanced BM25 search query."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    user_query = "best practices for training deep neural networks"
    
    # Step 1: Get top features by BM25 IDF
    top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
        es=es,
        index="ml_articles",
        field="body",
        query_text=user_query,
        top_k=5
    )
    
    print(f"Original query: {user_query}")
    print(f"Selected features: {top_features}\n")
    
    # Step 2: Build ES query with selected features
    # Option A: Simple multi-match with top features
    simple_query = {
        "query": {
            "multi_match": {
                "query": " ".join(top_features),
                "fields": ["title^2", "body"],
                "type": "best_fields"
            }
        }
    }
    
    # Option B: Bool query with IDF-based boosting
    boosted_query = {
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "body": {
                                "query": feature,
                                "boost": idf_scores[feature]  # Use IDF as boost
                            }
                        }
                    }
                    for feature in top_features
                ]
            }
        }
    }
    
    # Step 3: Execute search
    print("Executing enhanced BM25 search...")
    results = es.search(index="ml_articles", body=boosted_query, size=5)
    
    print(f"\nTop {len(results['hits']['hits'])} results:")
    for i, hit in enumerate(results['hits']['hits'], 1):
        print(f"  {i}. Score: {hit['_score']:.2f} - {hit['_source'].get('title', 'N/A')}")
    
    return results


# =============================================================================
# Example 4: Compare Original vs Feature-Selected Query
# =============================================================================

def example_compare_queries():
    """Compare search results: original query vs feature-selected query."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    original_query = "what are the best machine learning algorithms for text classification"
    
    # Get top features
    top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
        es=es,
        index="data_science_docs",
        field="text",
        query_text=original_query,
        top_k=4
    )
    
    print(f"Original query: {original_query}")
    print(f"Feature-selected query: {' '.join(top_features)}\n")
    
    # Search with original query
    results_original = es.search(
        index="data_science_docs",
        body={"query": {"match": {"text": original_query}}},
        size=3
    )
    
    # Search with feature-selected query
    results_filtered = es.search(
        index="data_science_docs",
        body={"query": {"match": {"text": " ".join(top_features)}}},
        size=3
    )
    
    print("Results with ORIGINAL query:")
    for i, hit in enumerate(results_original['hits']['hits'], 1):
        print(f"  {i}. Score: {hit['_score']:.2f}")
    
    print("\nResults with FEATURE-SELECTED query:")
    for i, hit in enumerate(results_filtered['hits']['hits'], 1):
        print(f"  {i}. Score: {hit['_score']:.2f}")


# =============================================================================
# Example 5: Dynamic Feature Selection Based on Query Length
# =============================================================================

def example_dynamic_top_k():
    """Dynamically adjust top-k based on query characteristics."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    queries = [
        "ML",  # Short query
        "machine learning classification",  # Medium query
        "how to build and train a convolutional neural network for image recognition"  # Long query
    ]
    
    for query in queries:
        # Dynamically set top-k: use 50% of terms, min 2, max 10
        estimated_terms = len(query.split())
        dynamic_k = max(2, min(10, estimated_terms // 2))
        
        top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
            es=es,
            index="tech_docs",
            field="content",
            query_text=query,
            top_k=dynamic_k
        )
        
        print(f"Query: {query}")
        print(f"  Terms analyzed: {meta['total_unique_terms']}")
        print(f"  Top-{dynamic_k} selected: {top_features}")
        print()


# =============================================================================
# Example 6: Query Expansion with Feature Selection
# =============================================================================

def example_query_expansion():
    """Combine feature selection with query expansion."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    original_query = "neural network training"
    
    # Step 1: Select top features
    top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
        es=es,
        index="research_db",
        field="abstract",
        query_text=original_query,
        top_k=3
    )
    
    print(f"Original query: {original_query}")
    print(f"Core features: {top_features}")
    
    # Step 2: Find similar terms using ES more_like_this
    expansion_query = {
        "query": {
            "more_like_this": {
                "fields": ["abstract"],
                "like": " ".join(top_features),
                "min_term_freq": 1,
                "max_query_terms": 5
            }
        },
        "size": 0,
        "aggs": {
            "related_terms": {
                "significant_terms": {
                    "field": "abstract.keyword",
                    "size": 5
                }
            }
        }
    }
    
    # Step 3: Execute and get expanded terms
    print("\nExpanded query terms:")
    print(f"  {', '.join(top_features)} + [related terms from corpus]")


# =============================================================================
# Example 7: Batch Processing for Multiple Queries
# =============================================================================

def example_batch_feature_selection():
    """Process multiple queries efficiently."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    queries = [
        "deep learning computer vision",
        "natural language processing transformers",
        "reinforcement learning robotics",
        "time series forecasting models"
    ]
    
    print("Batch Feature Selection Results:\n")
    
    all_results = {}
    for query in queries:
        top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
            es=es,
            index="ai_papers",
            field="title",
            query_text=query,
            top_k=3
        )
        all_results[query] = top_features
        
        print(f"Query: {query}")
        print(f"  Top features: {', '.join(top_features)}")
        print(f"  IDF scores: {[f'{idf_scores[f]:.2f}' for f in top_features]}")
        print()
    
    return all_results


# =============================================================================
# Example 8: Real-World Search Pipeline
# =============================================================================

def example_complete_search_pipeline():
    """Complete search pipeline with BM25 feature selection."""
    
    es = Elasticsearch(['http://localhost:9200'])
    
    # User's search query
    user_query = "python libraries for machine learning and data visualization"
    
    print("=" * 70)
    print("COMPLETE SEARCH PIPELINE")
    print("=" * 70)
    
    # Step 1: Tokenization and feature selection
    print(f"\n1. Original Query:\n   {user_query}")
    
    top_features, idf_scores, meta = select_top_k_features_by_bm25_idf(
        es=es,
        index="python_packages",
        field="description",
        query_text=user_query,
        top_k=5,
        min_idf=1.5
    )
    
    print(f"\n2. BM25 Tokenization & Analysis:")
    print(f"   Total terms analyzed: {meta['total_unique_terms']}")
    print(f"   Documents in corpus: {meta['N']:,}")
    
    print(f"\n3. Top-{len(top_features)} Features by IDF:")
    for i, feature in enumerate(top_features, 1):
        print(f"   {i}. {feature:20s} IDF={idf_scores[feature]:.4f}")
    
    # Step 2: Build and execute search
    search_query = {
        "query": {
            "bool": {
                "should": [
                    {"match": {"description": {"query": f, "boost": idf_scores[f]}}}
                    for f in top_features
                ],
                "minimum_should_match": 1
            }
        },
        "size": 10
    }
    
    print("\n4. Executing BM25 Search...")
    results = es.search(index="python_packages", body=search_query)
    
    print(f"\n5. Search Results (Top {len(results['hits']['hits'])}):")
    for i, hit in enumerate(results['hits']['hits'], 1):
        score = hit['_score']
        title = hit['_source'].get('name', 'N/A')
        print(f"   {i}. [{score:.2f}] {title}")
    
    return results


# =============================================================================
# Main
# =============================================================================

if __name__ == "__main__":
    print("=" * 70)
    print("ES BM25 TOKENIZATION + TOP-K FEATURE SELECTION")
    print("=" * 70)
    print()
    
    # Run examples (uncomment to try)
    # example_basic_top_k_selection()
    # example_with_min_idf()
    # example_enhanced_bm25_search()
    # example_compare_queries()
    # example_dynamic_top_k()
    # example_query_expansion()
    # example_batch_feature_selection()
    # example_complete_search_pipeline()
    
    print("\nNote: Ensure Elasticsearch is running with appropriate indices!")

In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

es.indices.create(
    index="docs",
    mappings={
        "properties": {
            "content": {
                "type": "text"   # default analyzer: standard, default similarity: BM25
            }
        }
    }
)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'docs'})

In [18]:
# ...existing code...
from elasticsearch import Elasticsearch, helpers  # ensure Elasticsearch is imported
import math
from typing import Dict, List, Tuple, Optional
# ...existing code...

def _bm25_idf(N: int, df: int) -> float:
    return math.log(1.0 + (N - df + 0.5) / (df + 0.5))

def _field_doc_count(es: Elasticsearch, index: str, field: str) -> int:
    """
    Prefer field-level doc_count (docs that actually contain terms for `field`).
    Falls back to total index docs if unavailable.
    """
    try:
        resp = es.termvectors(
            index=index,
            doc={field: ""},
            fields=[field],
            term_statistics=True,
            positions=False, offsets=False, payloads=False,
        )
    except TypeError:
        resp = es.termvectors(
            index=index,
            body={"doc": {field: ""}, "fields": [field], "term_statistics": True,
                  "positions": False, "offsets": False, "payloads": False}
        )
    tv = (resp or {}).get("term_vectors", {}).get(field, {}) or {}
    return int(tv.get("field_statistics", {}).get("doc_count", 0)) or 0

def _index_doc_count(es: Elasticsearch, index: str) -> int:
    stats = es.indices.stats(index=index, metric="docs")
    return int(stats["indices"][index]["primaries"]["docs"]["count"])

def iter_doc_ids(es: Elasticsearch, index: str, query: dict = {"match_all": {}}):
    for hit in helpers.scan(es, index=index, query={"query": query}, _source=False):
        yield hit["_id"]

def topk_tokens_by_idf(
    es: Elasticsearch,
    index: str,
    field: str,
    k: int = 50,
    doc_ids=None,
    mode: str = "global",          # "global" or "per_doc"
    batch_size: int = 256,
    min_df: int = 2,
    include_terms=None,
    exclude_terms=None,
    sample_max_docs: Optional[int] = None,  # optional cap to improve latency
):
    """
    mode="global": aggregate df per term (global rarity) and return the top-k rarest tokens.
    mode="per_doc": return a dict(doc_id -> top-k tokens by idf for that doc).
    """
    # Prefer field-aware N; fallback to total docs
    N = _field_doc_count(es, index, field) or _index_doc_count(es, index)
    if N == 0:
        return [] if mode == "global" else {}

    if doc_ids is None:
        doc_ids = list(iter_doc_ids(es, index))
        if sample_max_docs is not None and len(doc_ids) > sample_max_docs:
            doc_ids = doc_ids[:sample_max_docs]

    include_terms = set(include_terms or [])
    exclude_terms = set(exclude_terms or [])

    def term_ok(t: str) -> bool:
        if include_terms and t not in include_terms:
            return False
        if t in exclude_terms:
            return False
        return True

    if mode == "global":
        df_map: Dict[str, int] = {}
        for i in range(0, len(doc_ids), batch_size):
            chunk = doc_ids[i:i+batch_size]
            tv = es.mtermvectors(
                index=index,
                ids=chunk,
                fields=[field],
                term_statistics=True,
                field_statistics=False,
                offsets=False,
                positions=False,
                payloads=False,
            )
            for doc in tv.get("docs", []):
                if doc.get("found") is False:
                    continue
                terms = doc.get("term_vectors", {}).get(field, {}).get("terms", {}) or {}
                for t, stats in terms.items():
                    if not term_ok(t):
                        continue
                    df = int(stats.get("doc_freq", 0))
                    if df >= min_df:
                        df_map[t] = df  # global df (same across docs)

        items = [(t, _bm25_idf(N, df)) for t, df in df_map.items()]
        items.sort(key=lambda x: x[1], reverse=True)
        return items[:k]

    elif mode == "per_doc":
        out: Dict[str, List[Tuple[str, float]]] = {}
        for i in range(0, len(doc_ids), batch_size):
            chunk = doc_ids[i:i+batch_size]
            tv = es.mtermvectors(
                index=index,
                ids=chunk,
                fields=[field],
                term_statistics=True,
                field_statistics=False,
                offsets=False,
                positions=False,
                payloads=False,
            )
            for doc in tv.get("docs", []):
                if doc.get("found") is False:
                    continue
                doc_id = doc.get("_id")
                terms = doc.get("term_vectors", {}).get(field, {}).get("terms", {}) or {}
                scored: List[Tuple[str, float]] = []
                for t, stats in terms.items():
                    if not term_ok(t):
                        continue
                    df = int(stats.get("doc_freq", 0))
                    if df < min_df:
                        continue
                    scored.append((t, _bm25_idf(N, df)))
                scored.sort(key=lambda x: x[1], reverse=True)
                out[doc_id] = scored[:k]
        return out

    else:
        raise ValueError("mode must be 'global' or 'per_doc'")
# ...existing code...

In [19]:
_index_doc_count(model.es, model.index_name)

100

In [20]:
# 1) Top-100 rarest tokens across the index (by BM25-IDF):
top_global = topk_tokens_by_idf(model.es, index=model.index_name, field="content", k=100, mode="global", min_df=3)

# 2) For every document, return its top-20 high-IDF tokens:
top_per_doc = topk_tokens_by_idf(model.es, index=model.index_name, field="content", k=20, mode="per_doc", min_df=3)


In [21]:
top_global

[('accelerate', 3.3623575483458916),
 ('automation', 3.3623575483458916),
 ('continue', 3.3623575483458916),
 ('robotics', 3.3623575483458916),
 ('eating', 3.3623575483458916),
 ('improve', 3.3623575483458916),
 ('overall', 3.3623575483458916),
 ('banking', 3.3623575483458916),
 ('industry', 3.3623575483458916),
 ('investors', 3.3623575483458916),
 ('morning', 3.3623575483458916),
 ('optimism', 3.3623575483458916),
 ('showed', 3.3623575483458916),
 ('happening', 3.3623575483458916),
 ('rapid', 3.3623575483458916),
 ('technology', 3.3623575483458916),
 ('forecasts', 3.3623575483458916),
 ('rain', 3.3623575483458916),
 ('losses', 3.3623575483458916),
 ('slight', 3.3623575483458916),
 ('complex', 3.3623575483458916),
 ('computing', 3.3623575483458916),
 ('problems', 3.3623575483458916),
 ('promise', 3.3623575483458916),
 ('quantum', 3.3623575483458916),
 ('shows', 3.3623575483458916),
 ('solving', 3.3623575483458916),
 ('ai', 3.3623575483458916),
 ('deep', 3.3623575483458916),
 ('heavily'

In [23]:
from typing import List, Dict, Any, Tuple
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def _bm25_idf(df: np.ndarray, N: int, add_one: bool = True) -> np.ndarray:
    """
    Robertson–Spärck Jones IDF used by BM25.
      idf(t) = log( (N - df_t + 0.5) / (df_t + 0.5) ) [+ 1 optionally]
    add_one=True keeps all idf >= 0 (common in implementations).
    """
    idf = np.log((N - df + 0.5) / (df + 0.5))
    if add_one:
        idf = idf + 1.0
    return idf

def extract_topk_bm25idf_tfidf(
    docs: List[str],
    k: int = 10,
    *,
    ngram_range: Tuple[int, int] = (1, 1),
    stop_words: str | None = None,   # e.g. 'english'
    lowercase: bool = True,
    min_df: int | float = 1,
    max_df: int | float = 1.0,
    bm25_add_one: bool = True
) -> Dict[str, Any]:
    """
    For each document i:
      - Top-k tokens by BM25 IDF (restricted to tokens present in doc i)
      - Top-k tokens by TF-IDF (within doc i)
      - Intersection between those two top-k sets
    Also returns global BM25-IDF table for the whole corpus.

    Returns:
      {
        'per_doc': List[{
            'doc_index': i,
            'topk_bm25idf': List[Tuple[token, bm25_idf_value]],
            'topk_tfidf':   List[Tuple[token, tfidf_value]],
            'intersection': List[str]
        }, ...],
        'global_bm25idf': List[Tuple[token, bm25_idf_value_sorted_desc]]
      }
    """
    N = len(docs)

    # 1) Fit a CountVectorizer to get df (document frequencies) with the *exact* tokenization you want.
    count_vec = CountVectorizer(
        ngram_range=ngram_range,
        stop_words=stop_words,
        lowercase=lowercase,
        min_df=min_df,
        max_df=max_df
    )
    X_count = count_vec.fit_transform(docs)        # shape: [N_docs, V]
    vocab = count_vec.vocabulary_                  # token -> col index
    idx_to_tok = {j: t for t, j in vocab.items()}

    # Document frequency per term (how many docs the term appears in)
    # (X_count > 0) is sparse CSR boolean; sum(axis=0) -> df row vector
    df = np.asarray((X_count > 0).sum(axis=0)).ravel()

    # 2) Compute BM25 IDF vector (global, size V)
    bm25_idf_vec = _bm25_idf(df, N, add_one=bm25_add_one)

    # 3) Fit a TF-IDF vectorizer using the *same vocabulary* to keep tokenization aligned
    tfidf_vec = TfidfVectorizer(
        ngram_range=ngram_range,
        stop_words=stop_words,
        lowercase=lowercase,
        vocabulary=vocab,   # lock to same columns as CountVectorizer
        min_df=min_df,
        max_df=max_df,
        norm=None           # keep raw tf-idf weights (optional)
    )
    X_tfidf = tfidf_vec.fit_transform(docs)        # shape: [N_docs, V]

    per_doc_results = []
    for i in range(N):
        # Nonzero indices (tokens present) in doc i
        row = X_count.getrow(i)
        present_idx = row.indices  # indices of tokens present in doc i

        # --- Top-k by BM25-IDF (restricted to tokens present in this doc) ---
        if present_idx.size > 0:
            present_bm25 = bm25_idf_vec[present_idx]
            order_bm25 = np.argsort(-present_bm25)[:k]
            topk_bm25 = [(idx_to_tok[present_idx[j]], float(present_bm25[j])) for j in order_bm25]
        else:
            topk_bm25 = []

        # --- Top-k by TF-IDF (within this doc) ---
        row_tfidf = X_tfidf.getrow(i)
        if row_tfidf.nnz > 0:
            tfidf_idx = row_tfidf.indices
            tfidf_vals = row_tfidf.data
            order_tfidf = np.argsort(-tfidf_vals)[:k]
            topk_tfidf = [(idx_to_tok[tfidf_idx[j]], float(tfidf_vals[j])) for j in order_tfidf]
        else:
            topk_tfidf = []

        # --- Intersection of token *strings* ---
        set_bm25 = {t for t, _ in topk_bm25}
        set_tfidf = {t for t, _ in topk_tfidf}
        inter = sorted(set_bm25 & set_tfidf)

        per_doc_results.append({
            "doc_index": i,
            "topk_bm25idf": topk_bm25,
            "topk_tfidf": topk_tfidf,
            "intersection": inter
        })

    # 4) Global top-k by BM25-IDF across the corpus (no doc restriction)
    order_global = np.argsort(-bm25_idf_vec)
    global_bm25idf = [(idx_to_tok[j], float(bm25_idf_vec[j])) for j in order_global[:k]]

    return {
        "per_doc": per_doc_results,
        "global_bm25idf": global_bm25idf
    }

# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    docs = [
        "Basel III credit risk policy for retail loan portfolios.",
        "Internal compliance policy for AML and KYC procedures.",
        "Loan underwriting guidelines and credit scoring using FICO.",
        "General policy memo; not specific to credit risk."
    ]

    out = extract_topk_bm25idf_tfidf(
        docs,
        k=5,
        ngram_range=(1, 1),      # allow unigrams + bigrams (often helpful)
        stop_words='english',   # optional
        lowercase=True,
        min_df=1,
        max_df=1.0,
        bm25_add_one=True
    )

    # Pretty-print a sample
    from pprint import pprint
    print("\nGlobal BM25-IDF top terms:")
    pprint(out["global_bm25idf"])

    print("\nPer-document results:")
    for row in out["per_doc"]:
        print(f"\nDoc {row['doc_index']}:")
        print("  Top-k by BM25-IDF (present in doc):")
        pprint(row["topk_bm25idf"])
        print("  Top-k by TF-IDF:")
        pprint(row["topk_tfidf"])
        print("  Intersection:")
        pprint(row["intersection"])



Global BM25-IDF top terms:
[('aml', 1.8472978603872037),
 ('specific', 1.8472978603872037),
 ('scoring', 1.8472978603872037),
 ('retail', 1.8472978603872037),
 ('procedures', 1.8472978603872037)]

Per-document results:

Doc 0:
  Top-k by BM25-IDF (present in doc):
[('basel', 1.8472978603872037),
 ('iii', 1.8472978603872037),
 ('portfolios', 1.8472978603872037),
 ('retail', 1.8472978603872037),
 ('loan', 1.0)]
  Top-k by TF-IDF:
[('basel', 1.916290731874155),
 ('iii', 1.916290731874155),
 ('portfolios', 1.916290731874155),
 ('retail', 1.916290731874155),
 ('loan', 1.5108256237659907)]
  Intersection:
['basel', 'iii', 'loan', 'portfolios', 'retail']

Doc 1:
  Top-k by BM25-IDF (present in doc):
[('aml', 1.8472978603872037),
 ('compliance', 1.8472978603872037),
 ('internal', 1.8472978603872037),
 ('kyc', 1.8472978603872037),
 ('procedures', 1.8472978603872037)]
  Top-k by TF-IDF:
[('aml', 1.916290731874155),
 ('compliance', 1.916290731874155),
 ('internal', 1.916290731874155),
 ('kyc', 1

In [ ]:
from typing import Iterable, Tuple, Set, Dict, List, Optional
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
_WORD = r"[A-Za-z]+(?:[-'][A-Za-z]+)*"          # handles hyphenated words & apostrophes: policy-maker, bank's
_PLACEHOLDER = r"__PHRASE_\d+__"
_PUNCT = r"[^\w\s]"                             # any single non-word, non-space (.,;:!?()[]{}"”’ etc.)
_NUMBER = r"\d+(?:[.,]\d+)*%?|\$\d+(?:[.,]\d+)*"
# _TOKEN_PATTERN = re.compile(fr"{_PLACEHOLDER}|{_WORD}|{_PUNCT}|{_NUMBER}")
_TOKEN_PATTERN = re.compile(fr"{_PLACEHOLDER}|{_WORD}|{_NUMBER}")

def my_tokenizer(text):
    # Example: split on alphabetic words and hyphenated forms
    return re.findall(_TOKEN_PATTERN, text.lower())


def bm25_idf(df: np.ndarray, N: int, add_one: bool = False) -> np.ndarray:
    """
    Robertson–Spärck Jones IDF (used by BM25):
      idf(t) = log( (N - df_t + 0.5) / (df_t + 0.5) )
    If add_one=True, add +1 to keep non-negativity (some implementations do this).
    """
    idf = np.log((N - df + 0.5) / (df + 0.5))
    if add_one:
        idf = idf + 1.0
    return idf

def prune_vocab_by_bm25_idf(
    docs: Iterable[str],
    *,
    ngram_range: Tuple[int, int] = (1, 1),
    stop_words: Optional[str] = None,   # e.g., 'english'
    lowercase: bool = True,
    min_df: int | float = 1,            # you can still apply DF gates
    max_df: int | float = 1.0,
    add_one: bool = False,              # BM25-IDF + 1 or not
    floor_nonnegative: bool = True,     # drop tokens with negative BM25-IDF
    keep_top_k: Optional[int] = None,   # keep exactly top-k by BM25-IDF
    idf_percentile: Optional[float] = 90.0,  # or keep tokens >= this percentile
    whitelist: Optional[Iterable[str]] = None,
    blacklist: Optional[Iterable[str]] = None,
) -> Dict[str, object]:
    """
    Returns:
      {
        'kept_vocab': Set[str],
        'bm25_idf_table': List[(token, bm25_idf_value)]  # sorted desc
      }
    """
    docs = list(docs)
    N = len(docs)

    # 1) CountVectorizer to get DF with desired tokenization
    cv = CountVectorizer(
        ngram_range=ngram_range,
        stop_words=stop_words,
        lowercase=lowercase,
        min_df=min_df,
        max_df=max_df,
        # tokenizer=my_tokenizer
    )
    X = cv.fit_transform(docs)                # [N x V] sparse
    vocab = np.array(cv.get_feature_names_out())
    # print(vocab)
    df = np.asarray((X > 0).sum(axis=0)).ravel()  # doc frequency per term

    # 2) BM25-IDF
    idf_vals = bm25_idf(df, N, add_one=False)

    # 3) Base keep mask
    keep_mask = np.ones_like(idf_vals, dtype=bool)

    if floor_nonnegative and not add_one:
        # Drop very common terms whose BM25-IDF < 0
        keep_mask &= (idf_vals >= 0)

    # 4) Threshold by either top-k or percentile (if provided)
    #    (Apply after negative filter so ranking is among plausible terms.)
    candidate_idx = np.where(keep_mask)[0]
    if candidate_idx.size == 0:
        return {"kept_vocab": set(), "bm25_idf_table": []}

    cand_idf = idf_vals[candidate_idx]
    if keep_top_k is not None:
        print(keep_top_k)
        order = candidate_idx[np.argsort(-cand_idf)[:keep_top_k]]
        keep_mask = np.zeros_like(keep_mask, dtype=bool)
        keep_mask[order] = True
    elif idf_percentile is not None:
        thr = np.percentile(cand_idf, idf_percentile)
        keep_mask &= (idf_vals >= thr)

    kept_tokens = set(vocab[keep_mask])

    # 5) Apply whitelist / blacklist
    if whitelist:
        kept_tokens |= set(whitelist)
    if blacklist:
        kept_tokens -= set(blacklist)

    # 6) Produce a sorted table (useful for inspection/governance)
    order_all = np.argsort(-idf_vals)
    bm25_table = [(vocab[i], float(idf_vals[i])) for i in order_all]

    return {"kept_vocab": kept_tokens, "bm25_idf_table": bm25_table}



# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    docs = [
        "Basel III credit risk policy for retail loan portfolios.",
        "Internal compliance policy for AML and KYC procedures.",
        "Loan underwriting guidelines and credit scoring using FICO.",
        "General policy memo; not specific to credit risk."
    ]

    result = prune_vocab_by_bm25_idf(
        docs,
        ngram_range=(1,1),
        stop_words='english',
        add_one=False,              # classic BM25-IDF (can be negative)
        floor_nonnegative=True,     # drop negatives (very common terms)
        keep_top_k=10,
             # or set idf_percentile=85.0 instead
    )

    kept = result["kept_vocab"]
    table = result["bm25_idf_table"]  # peek top-10
    print("Kept vocab size:", len(kept))
    print(len(table))


10
Kept vocab size: 10
21


In [48]:
kept

{'aml',
 'internal',
 'memo',
 'portfolios',
 'procedures',
 'retail',
 'scoring',
 'specific',
 'underwriting',
 'using'}

In [52]:
set(kept).intersection(set(topk_vocab_tf_idf))

{'memo', 'portfolios', 'procedures', 'retail', 'scoring', 'specific'}

In [53]:
topk_vocab_tf_idf

{'credit',
 'loan',
 'memo',
 'policy',
 'portfolios',
 'procedures',
 'retail',
 'risk',
 'scoring',
 'specific'}

In [45]:
len(table)

21

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

def vocab_prune_by_df(docs, min_df=2, max_df=0.9, ngram_range=(1,1), stop_words='english'):
    cv = CountVectorizer(min_df=min_df, max_df=max_df,
                         ngram_range=ngram_range, stop_words=stop_words)
    X = cv.fit_transform(docs)               # N x V
    vocab = np.array(cv.get_feature_names_out())
    df = np.asarray((X > 0).sum(axis=0)).ravel()
    keep_mask = (df >= (min_df if isinstance(min_df,int) else min_df*X.shape[0])) & \
                (df <= (max_df if isinstance(max_df,int) else max_df*X.shape[0]))
    return set(vocab[keep_mask])

def vocab_prune_by_idf(docs, keep_top_k=None, idf_percentile=90, ngram_range=(1,1)):
    tfidf = TfidfVectorizer(ngram_range=ngram_range, stop_words='english', norm=None)
    X = tfidf.fit_transform(docs)
    vocab = np.array(tfidf.get_feature_names_out())
    idf = tfidf.idf_                         # length V
    if keep_top_k:
        keep_idx = np.argsort(-idf)[:keep_top_k]
    else:
        thr = np.percentile(idf, idf_percentile)
        keep_idx = np.where(idf >= thr)[0]
    return set(vocab[keep_idx])

def vocab_prune_by_global_tfidf(docs, keep_top_k=20000, ngram_range=(1,1)):
    tfidf = TfidfVectorizer(ngram_range=ngram_range, stop_words='english', norm=None, tokenizer=my_tokenizer)
    X = tfidf.fit_transform(docs)            # N x V (sparse)
    vocab = np.array(tfidf.get_feature_names_out())
    # Mean TF-IDF across docs as an importance proxy
    importance = np.asarray(X.mean(axis=0)).ravel()
    keep_idx = np.argsort(-importance)[:keep_top_k]
    return set(vocab[keep_idx])


In [256]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

text = "The bank's co-op site is https://wf.com. Call 1-800-555-1234!"
tokens = es.indices.analyze(body={"analyzer": "standard", "text": text})
print([t["token"] for t in tokens["tokens"]])


['the', "bank's", 'co', 'op', 'site', 'is', 'https', 'wf.com', 'call', '1', '800', '555', '1234']
